In [1]:
%run "get_packages.py"

C:\Program64\Python\Python\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
today = date.today()
end_date = today.strftime("%Y%m%d")
strt_date="20241030"
std_size = 0.03
concentration_size = 0.05
step_size = 0.005
TE_upper_bound = 0.012
TE_lower_bound = 0.008
features_weight = 0.33
multiplier = 2
carry_cost_lim = 0.003
start_date = 1/1/2003

REER_lag = 22
TB_lag = 30
GDP_lag = 120
CPI_lag = 30

Adj_REER = ["REER", "Conditional REER", "Symmetrical REER"]
Carry = ["Nominal Carry", "Real Carry"]
Conviction = ["Awa", "Agn"]

currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]


In [135]:
#SPOT PRICE
from blp import blp
bquery = blp.BlpQuery().start()
spot_query = bquery.bdh(
        ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy", "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy", "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy", "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy", "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy", "KRW CMPN Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

spot_query['security_short'] = spot_query['security'].str[:3]

spot = spot_query.pivot_table(index='date', columns='security_short', values='PX_LAST', aggfunc='first')
spot.reset_index( inplace=True)
spot.rename(columns={'date':'DATE'}, inplace=True)
spot['DATE'] = pd.to_datetime(spot['DATE'])
spot.sort_values(by='DATE', ascending=False, inplace=True)
spot.dropna(inplace=True)

# Extract relevant columns
columns = ["DATE"] + currencies
spot_subset = spot[columns]

# Calculate daily returns
spot_subset.set_index("DATE", inplace=True)
daily_return = spot_subset.shift(-1)/spot_subset  -1
daily_return.reset_index(inplace=True)
#daily_return.dropna(inplace=True)
spot_return = daily_return.melt(id_vars=["DATE"], var_name="Currency", value_name="Daily_Return")


In [136]:
bquery = blp.BlpQuery().start()
MA_query = bquery.bdh(
        ["BRL Curncy","CLP Curncy","CNY Curncy","COP Curncy","CZK Curncy","HUF Curncy","IDR Curncy","MYR Curncy","MXN Curncy","PEN Curncy","PHP Curncy","PLN Curncy","RON Curncy","RUB Curncy","ZAR Curncy","THB Curncy","TRY Curncy","INR Curncy","SGD Curncy","TWD Curncy","KRW Curncy"],
        ["MOV_AVG_5D","MOV_AVG_20D"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})
MA_query.rename(columns={'date':'DATE'}, inplace=True)
MA_query['Currency'] = MA_query['security'].str[:3]
MA_query.drop(columns="security", inplace=True)
MA_query['Momentum'] = np.where(MA_query['MOV_AVG_5D'] > MA_query['MOV_AVG_20D'], 1, -1)
Momentum = MA_query[["DATE", "Currency", "MOV_AVG_5D", "MOV_AVG_20D","Momentum"]]
Momentum['DATE'] = pd.to_datetime(Momentum['DATE'])
Momentum.sort_values(by='DATE', ascending=False, inplace=True)
Momentum.reset_index(drop=True, inplace=True)
#Momentum.dropna(inplace=True)
Momentum

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)


,DATE,Currency,MOV_AVG_5D,MOV_AVG_20D,Momentum
0,2024-11-27,KRW,1398.5400,1394.5200,1
1,2024-11-27,PLN,4.1186,4.0734,1
2,2024-11-27,CZK,24.1299,23.7800,1
3,2024-11-27,THB,34.6290,34.4230,1
4,2024-11-27,PEN,3.7738,3.7810,-1
...,...,...,...,...,...
436,2024-10-30,IDR,15682.0000,15590.0000,1
437,2024-10-30,PHP,58.1870,57.3620,1
438,2024-10-30,PLN,4.0168,3.9684,1
439,2024-10-30,CNY,7.1232,7.0937,1


In [341]:
print(Momentum)

          DATE Currency  MOV_AVG_5D  MOV_AVG_20D  Momentum
0   2024-11-27      KRW   1398.5400    1394.5200         1
1   2024-11-27      PLN      4.1186       4.0734         1
2   2024-11-27      CZK     24.1299      23.7800         1
3   2024-11-27      THB     34.6290      34.4230         1
4   2024-11-27      PEN      3.7738       3.7810        -1
..         ...      ...         ...          ...       ...
436 2024-10-30      IDR  15682.0000   15590.0000         1
437 2024-10-30      PHP     58.1870      57.3620         1
438 2024-10-30      PLN      4.0168       3.9684         1
439 2024-10-30      CNY      7.1232       7.0937         1
440 2024-10-30      BRL      5.7212       5.6342         1

[441 rows x 5 columns]


In [342]:


def create_momentum_dict(momentum_df):
    """Creates a dictionary of DataFrames with momentum data organized by date.

    Args:
        momentum_df: The input DataFrame with 'DATE', 'Currency', 'MOV_AVG_5D', 'MOV_AVG_20D', and 'Momentum' columns.

    Returns:
        A dictionary where keys are dates (YYYYMMDD strings) and values are DataFrames
        with 'Currency' as index and 'MOV_AVG_5D', 'MOV_AVG_20D', and 'Momentum' as columns.
    """

    momentum_dict = {}
    for date, date_df in momentum_df.groupby('DATE'):
        date_str = date.strftime('%Y%m%d')
        
        if date_str not in momentum_dict: #Create empty dataframes for dates to avoid errors if data is missing.
            momentum_dict[date_str] = pd.DataFrame(index = currencies, columns=['MOV_AVG_5D', 'MOV_AVG_20D', 'Momentum'])



        temp_df = date_df.set_index('Currency')


        temp_df = temp_df.reindex(currencies)




        momentum_dict[date_str] = temp_df #


    return momentum_dict


momentum_dict = create_momentum_dict(Momentum) #

In [347]:
momentum_dict['20241030']

,DATE,MOV_AVG_5D,MOV_AVG_20D,Momentum
Currency,,,,
BRL,2024-10-30,5.7212,5.6342,1.0
CLP,2024-10-30,951.9300,940.1000,1.0
CNY,2024-10-30,7.1232,7.0937,1.0
COP,2024-10-30,4354.0700,4265.0200,1.0
CZK,2024-10-30,23.3959,23.2463,1.0
HUF,2024-10-30,374.0700,369.2900,1.0
IDR,2024-10-30,15682.0000,15590.0000,1.0
MYR,2024-10-30,4.3619,4.3096,1.0
MXN,2024-10-30,20.0117,19.7017,1.0


In [138]:
#3M Implied vol
from blp import blp 
bquery = blp.BlpQuery().start()
IMPVOL_query = bquery.bdh(
        ["USDBRLV3M Curncy","USDCLPV3M Curncy","USDCNYV3M Curncy","USDCOPV3M Curncy","USDCZKV3M Curncy","USDHUFV3M Curncy","USDIDRV3M Curncy","USDMYRV3M Curncy","USDMXNV3M Curncy","USDPENV3M Curncy","USDPHPV3M Curncy","USDPLNV3M Curncy","USDRONV3M Curncy","USDRUBV3M Curncy","USDZARV3M Curncy","USDTHBV3M Curncy","USDTRYV3M Curncy","USDINRV3M Curncy","USDSGDV3M Curncy","USDTWDV3M Curncy","USDKRWV3M Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"}) #

IMPVOL_query.rename(columns={'date':'DATE', 'PX_LAST':'3M Implied Vol'}, inplace=True)

conditions = [
    (IMPVOL_query['security'] == 'USDBRLV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCLPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCNYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCOPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCZKV3M Curncy'),
    (IMPVOL_query['security'] == 'USDHUFV3M Curncy'),
    (IMPVOL_query['security'] == 'USDIDRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMYRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMXNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPENV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPHPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPLNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRONV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRUBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDZARV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTHBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTRYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDINRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDSGDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTWDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDKRWV3M Curncy'),
]

IMPVOL_query['Currency'] = np.select(conditions, currencies, default=None)
IMPVOL_query.drop(columns="security", inplace=True)
IMPVOL_query.sort_values(by='DATE', ascending=False, inplace=True)
IMPVOL_query=IMPVOL_query[['DATE','Currency','3M Implied Vol']]
IMPVOL_query

,DATE,Currency,3M Implied Vol
440,2024-11-27,MYR,6.4625
430,2024-11-27,ZAR,13.3200
420,2024-11-27,KRW,9.1150
421,2024-11-27,CNY,6.0150
422,2024-11-27,RON,7.4850
...,...,...,...
17,2024-10-30,PEN,5.5125
18,2024-10-30,RUB,24.2300
19,2024-10-30,ZAR,14.1950
20,2024-10-30,SGD,6.2650


In [345]:

def create_volatility_dict(volatility_df):
    """Creates a dictionary of DataFrames with volatility data organized by date.

    Args:
        volatility_df: Input DataFrame with 'DATE', 'Currency', and '3M Implied Vol' columns.

    Returns:
        A dictionary with dates ('YYYYMMDD') as keys and DataFrames as values.
        DataFrames have 'Currency' as index and '3M Implied Vol' as column.
    """

    volatility_dict = {}

    for date, date_df in volatility_df.groupby('DATE'):  # Group by date
        date_str = date.strftime('%Y%m%d')

        if date_str not in volatility_dict:
             volatility_dict[date_str] = pd.DataFrame(index=currencies, columns=['3M Implied Vol'])



        temp_df = date_df.set_index('Currency') # Set Currency as index

        temp_df = temp_df.reindex(currencies)  # Reindex to ensure all currencies and order


        volatility_dict[date_str] = temp_df


    return volatility_dict

volatility_dict = create_volatility_dict(IMPVOL_query)


In [346]:
volatility_dict

{'20241030':                DATE  3M Implied Vol
 Currency                           
 BRL      2024-10-30         15.6820
 CLP      2024-10-30         13.8900
 CNY      2024-10-30          7.2950
 COP      2024-10-30         14.2475
 CZK      2024-10-30          8.4550
 HUF      2024-10-30         11.1775
 IDR      2024-10-30          8.2550
 MYR      2024-10-30          7.1475
 MXN      2024-10-30         18.1450
 PEN      2024-10-30          5.5125
 PHP      2024-10-30          7.2125
 PLN      2024-10-30         10.1150
 RON      2024-10-30          7.4700
 RUB      2024-10-30         24.2300
 ZAR      2024-10-30         14.1950
 THB      2024-10-30          9.7725
 TRY      2024-10-30         16.5100
 INR      2024-10-30          2.5725
 SGD      2024-10-30          6.2650
 TWD      2024-10-30          8.6000
 KRW      2024-10-30         11.0800
 UAH             NaT             NaN
 EGP             NaT             NaN
 EUR             NaT             NaN
 JPY             NaT      

In [139]:
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
from blp import blp 
all_currencies = ["BRL","CLP","CNY","COP","CZK","HUF","IDR","MYR","MXN","PEN","PHP","PLN","RON","RUB","ZAR","THB","TRY","INR","SGD","TWD","KRW","UAH","EGP","EUR","JPY","USD","GBP","CAD","ARS","CHF"]

# Create an empty list to store the dataframes
all_cpi_data = []

# Loop through the dates from 2020 to today
start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('today') 
for date in pd.date_range(start=start_date, end=end_date, freq='MS'): # 'MS' for month start frequency
    specific_date = date + pd.DateOffset(months=1) - pd.DateOffset(days=1)  # Get the last day of the month

    begin = ((specific_date - pd.DateOffset(months=2)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')
    end = ((specific_date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')

    bquery = blp.BlpQuery().start()
    CPI_query = bquery.bdh(
            ["BZPIIPCM Index","CLINNSMO Index","CNCPIMOM Index","COCPIMOM Index","CZCPMOM Index","HUCPIMM Index","IDCPIM Index","MACPIMOM Index","MXCPCHNG Index","PRCPNONC Index","PHC2MOM Index","POCPIMOM Index","ROCOPMOM Index","RUCPIMOM Index","SACPIMOM Index","THCPIMOM Index","TUCPIM Index","INFUTOT Index","SICPIMOM Index","TWCPIMOM Index","KOCPIMOM Index","UACPTMOM Index","EGCPMOM Index","ECCPEMUM Index","JNCPIMOM Index","CPURNSA% Index","UKRPCJMR Index","CACPICHG Index","ARNCNINX Index","SZCPIMOM Index"],
            ["PX_LAST"],
            start_date= begin,
            end_date= end, 
            options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"MONTHLY", 'nonTradingDayFillMethod':'NIL_VALUE'})
    CPI_query.rename(columns={'PX_LAST':'CPI'}, inplace=True)

    conditions = [
    (CPI_query['security'] == 'BZPIIPCM Index'),
    (CPI_query['security'] == 'CLINNSMO Index'),
    (CPI_query['security'] == 'CNCPIMOM Index'),
    (CPI_query['security'] == 'COCPIMOM Index'),
    (CPI_query['security'] == 'CZCPMOM Index'),
    (CPI_query['security'] == 'HUCPIMM Index'),
    (CPI_query['security'] == 'IDCPIM Index'),
    (CPI_query['security'] == 'MACPIMOM Index'),
    (CPI_query['security'] == 'MXCPCHNG Index'),
    (CPI_query['security'] == 'PRCPNONC Index'),
    (CPI_query['security'] == 'PHC2MOM Index'),
    (CPI_query['security'] == 'POCPIMOM Index'),
    (CPI_query['security'] == 'ROCOPMOM Index'),
    (CPI_query['security'] == 'RUCPIMOM Index'),
    (CPI_query['security'] == 'SACPIMOM Index'),
    (CPI_query['security'] == 'THCPIMOM Index'),
    (CPI_query['security'] == 'TUCPIM Index'),
    (CPI_query['security'] == 'INFUTOT Index'),
    (CPI_query['security'] == 'SICPIMOM Index'),
    (CPI_query['security'] == 'TWCPIMOM Index'),
    (CPI_query['security'] == 'KOCPIMOM Index'),
    (CPI_query['security'] == 'UACPTMOM Index'),
    (CPI_query['security'] == 'EGCPMOM Index'),
    (CPI_query['security'] == 'ECCPEMUM Index'),
    (CPI_query['security'] == 'JNCPIMOM Index'),
    (CPI_query['security'] == 'CPURNSA% Index'),
    (CPI_query['security'] == 'UKRPCJMR Index'),
    (CPI_query['security'] == 'CACPICHG Index'),
    (CPI_query['security'] == 'ARNCNINX Index'),
    (CPI_query['security'] == 'SZCPIMOM Index')
    ]

    CPI_query['Currency'] = np.select(conditions, all_currencies, default=None)
    CPI_query.loc[CPI_query['Currency'] == 'INR', 'CPI'] = 100*(CPI_query['CPI']/185.8 -1)
        
    CPI_query=CPI_query[['Currency','CPI']]
    CPI_query['Date'] = specific_date # Add a Date column

    all_cpi_data.append(CPI_query) # Append the dataframe to the list


# Concatenate all dataframes vertically
final_cpi_data = pd.concat(all_cpi_data, ignore_index=True)

print(final_cpi_data)
CPI_query = final_cpi_data.copy()

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)
C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)
C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determinin

     Currency      CPI       Date
0         MXN  1.03000 2018-01-31
1         UAH  0.90000 2018-01-31
2         KRW -0.70000 2018-01-31
3         TWD  0.24000 2018-01-31
4         SGD  0.60000 2018-01-31
...       ...      ...        ...
2485      TWD  0.11000 2024-11-30
2486      SGD  0.30000 2024-11-30
2487      INR  4.52099 2024-11-30
2488      IDR -0.12000 2024-11-30
2489      MYR  0.00000 2024-11-30

[2490 rows x 3 columns]


In [140]:

def create_cpi_dict(final_cpi_data, all_currencies):
    cpi_dict = {}
    final_cpi_data['Date'] = pd.to_datetime(final_cpi_data['Date'])
    for date in final_cpi_data['Date'].unique():
        date_str = date.strftime('%Y-%m-%d')  # Format date as string for consistency
        date_data = final_cpi_data[final_cpi_data['Date'] == date].copy()

        # Create the correctly formatted DataFrame
        temp_df = pd.DataFrame(index=all_currencies, columns=['CPI']) # Use all_currencies as index
        for currency in all_currencies:  
            if (date_data['Currency'] == currency).any():  # Check if the currency exists for this date
                temp_df.loc[currency, 'CPI'] = date_data.loc[date_data['Currency'] == currency, 'CPI'].iloc[0]
            else:
                temp_df.loc[currency, 'CPI'] = np.nan  # Handle missing currencies
        temp_df = temp_df.reset_index().rename(columns={'index':'Currency'}) 
        cpi_dict[date_str] = temp_df  # Store with date string as key

    return cpi_dict
all_currencies = ["BRL","CLP","CNY","COP","CZK","HUF","IDR","MYR","MXN","PEN","PHP","PLN","RON","RUB","ZAR","THB","TRY","INR","SGD","TWD","KRW","UAH","EGP","EUR","JPY","USD","GBP","CAD","ARS","CHF"]
cpi_dict = create_cpi_dict(final_cpi_data, all_currencies)

In [141]:
cpi_dict['2018-01-31']
# fill cpi dataframe daily 


,Currency,CPI
0,BRL,0.28
1,CLP,0.1
2,CNY,0.0
3,COP,0.18
4,CZK,0.1
5,HUF,0.4
6,IDR,0.2
7,MYR,0.7
8,MXN,1.03
9,PEN,-0.19


In [26]:
from pandas.tseries.offsets import MonthEnd

# Calculate the start and end dates for the entire period (month-end frequency)
start_date = final_cpi_data['Date'].min()
end_date = final_cpi_data['Date'].max()

# Create a daily date range for the entire period
daily_dates = pd.date_range(start=start_date - pd.DateOffset(days=1), end=end_date, freq='D')

# Create an empty dataframe for the daily CPI data
final_daily_cpi_data = pd.DataFrame(columns=['Date', 'Currency', 'CPI'])

# Loop through each month-end date in the CPI_query
for month_end_date in final_cpi_data['Date'].unique():
    # Filter the CPI_query for the current month-end
    month_data = final_cpi_data[final_cpi_data['Date'] == month_end_date]

    # Get the start and end dates for the current month
    start_of_month = month_end_date.replace(day=1)

    # Filter the daily_dates for the current month
    current_month_dates = daily_dates[(daily_dates >= start_of_month) & (daily_dates <= month_end_date)]

    # Repeat the month's CPI data for each day in the month (using cross-join)
    repeated_data = pd.merge(pd.DataFrame({'Date': current_month_dates}), month_data, how='cross')

    # Append the repeated data to the final dataframe
    final_daily_cpi_data = pd.concat([final_daily_cpi_data, repeated_data], ignore_index=True)

# Sort the final dataframe by date
final_daily_cpi_data = final_daily_cpi_data.sort_values('Date').reset_index(drop=True)

print(final_daily_cpi_data)


C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_32256/2254841379.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_daily_cpi_data = pd.concat([final_daily_cpi_data, repeated_data], ignore_index=True)


      Date Currency       CPI     Date_x     Date_y
0      NaN      TRY  3.510000 2022-01-30 2022-01-31
1      NaN      RUB  0.960000 2022-01-30 2022-01-31
2      NaN      RON  0.000000 2022-01-30 2022-01-31
3      NaN      PLN  1.000000 2022-01-30 2022-01-31
4      NaN      PHP  0.600000 2022-01-30 2022-01-31
...    ...      ...       ...        ...        ...
30175  NaN      SGD  0.700000 2024-10-31 2024-10-31
30176  NaN      INR  3.875135 2024-10-31 2024-10-31
30177  NaN      TRY  2.470000 2024-10-31 2024-10-31
30178  NaN      IDR -0.030000 2024-10-31 2024-10-31
30179  NaN      MYR  0.100000 2024-10-31 2024-10-31

[30180 rows x 5 columns]


In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from pickle import TRUE
from dateutil.relativedelta import relativedelta
import os
from skopt import BayesSearchCV  # need scikit-optimize
from skopt.space import Real, Integer
import xbbg
import blp
from sklearn import preprocessing
import openpyxl
import plotly.express as px
import statsmodels
from xbbg import blp
import datetime
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import BDay

In [72]:
begin = ((date - pd.DateOffset(months=62)).replace(day=1)).strftime('%Y-%m-%d') # Start_date shifted by 62months
end = date.strftime('%Y-%m-%d')
    
REER_query = blp.bdh(  # Use bdh for historical data
    securities,
    ["PX_LAST"],
    start_date=begin,
    end_date=end # Add options if needed
)
REER_query.columns

MultiIndex([('BISBBRR Index', 'PX_LAST'),
            ('BISBCLR Index', 'PX_LAST'),
            ('BISBCNR Index', 'PX_LAST'),
            ('BISBCOP Index', 'PX_LAST'),
            ('BISBCZR Index', 'PX_LAST'),
            ('BISBHUR Index', 'PX_LAST'),
            ('BISBIDR Index', 'PX_LAST'),
            ('BISBMYR Index', 'PX_LAST'),
            ('BISBMXR Index', 'PX_LAST'),
            ('BISBPER Index', 'PX_LAST'),
            ('BISBPHR Index', 'PX_LAST'),
            ('BISBPLR Index', 'PX_LAST'),
            ('BISBROR Index', 'PX_LAST'),
            ('BISBRUR Index', 'PX_LAST'),
            ('BISBZAR Index', 'PX_LAST'),
            ('BISBTHR Index', 'PX_LAST'),
            ('BISBTRR Index', 'PX_LAST'),
            ('BISBINR Index', 'PX_LAST'),
            ('BISBSGR Index', 'PX_LAST'),
            ('BISBTWR Index', 'PX_LAST'),
            ('BISBKRR Index', 'PX_LAST')],
           )

In [73]:
reer_data = reer_data.reset_index(drop=True)  # Remove current index if date isn't a column


In [8]:
from xbbg import blp

def get_reer_data(date, securities, currencies):
    begin = ((date - pd.DateOffset(months=150)).replace(day=1)).strftime('%Y-%m-%d')
    end = date.strftime('%Y-%m-%d')

    REER_query = blp.bdh(
        securities,
        ["PX_LAST"],
        start_date=begin,
        end_date=end
    )

    REER_query.columns = REER_query.columns.get_level_values(0)  # Flatten MultiIndex

    ticker_to_currency = dict(zip(securities, currencies))
    REER_query = REER_query.rename(columns=ticker_to_currency)  #Rename currency tickers

    REER_query = REER_query.stack().reset_index() #Stacking to multiindex
    REER_query.columns = ['date', 'Currency', 'PX_LAST'] #Renaming accordingly because of stack's naming convention for the tickers columns
   
    REER_query = REER_query.pivot(index='date', columns='Currency', values='PX_LAST') #pivoting correctly after having stacked

    return REER_query


# Example Usage (replace with your lists and date)
securities = ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"]
currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]
date = pd.to_datetime('2024-11-26')

reer_data = get_reer_data(date, securities, currencies)
reer_data = reer_data.reset_index(drop=False)  # Remove current index if date isn't a column


In [275]:
print(reer_data)

Currency        date     BRL     CLP    CNY     COP     CZK     HUF     IDR  \
0         2012-04-30  162.58  120.54  86.79  144.74  100.21  112.14  107.45   
1         2012-05-31  154.71  119.69  87.53  146.22   98.00  111.96  106.81   
2         2012-06-30  151.64  118.77  87.40  148.25   96.79  111.99  107.06   
3         2012-07-31  154.24  122.09  87.33  148.37   96.68  113.96  107.40   
4         2012-08-31  153.32  124.04  86.98  145.26   97.78  116.60  106.52   
..               ...     ...     ...    ...     ...     ...     ...     ...   
145       2024-05-31  113.51  103.17  91.07  114.28  121.74  110.40  100.08   
146       2024-06-30  108.90  102.91  91.17  110.93  121.41  108.00   98.62   
147       2024-07-31  105.51  102.00  91.29  110.87  119.61  109.33   98.55   
148       2024-08-31  103.74  101.65  91.28  109.20  120.36  108.66   98.95   
149       2024-09-30  103.71  101.29  91.33  105.84  120.15  108.43  100.19   

Currency     INR    KRW  ...     PEN     PHP     PL

In [5]:

def calculate_5yr_avg_reer(reer_data, currency='BRL'):
    reer_data['date'] = pd.to_datetime(reer_data['date'])
    reer_data = reer_data.sort_values('date')  # Sort by date
    days_in_5_years = 365 * 5 + 1 # accounting for leap year, there could be two leap years even, this assumes 1
    reer_data[f'{currency}_5yr_rolling'] = (
        reer_data[currency].rolling(window=days_in_5_years, min_periods=60, center=False).mean()

    )
    return reer_data

reer_data = calculate_5yr_avg_reer(reer_data, currency='BRL')
print(reer_data.tail())

Currency       date     BRL     CLP    CNY     COP     CZK     HUF     IDR  \
145      2024-05-31  113.51  103.17  91.07  114.28  121.74  110.40  100.08   
146      2024-06-30  108.90  102.91  91.17  110.93  121.41  108.00   98.62   
147      2024-07-31  105.51  102.00  91.29  110.87  119.61  109.33   98.55   
148      2024-08-31  103.74  101.65  91.30  109.20  120.36  108.66   98.97   
149      2024-09-30  103.72  101.30  91.39  105.83  120.16  108.45  100.24   

Currency     INR    KRW  ...     PHP     PLN     RON     RUB     SGD    THB  \
145       101.22  95.16  ...  101.60  121.48  111.64  100.91  115.55  91.99   
146       102.77  94.22  ...  100.51  120.13  111.67  105.21  115.57  91.97   
147       103.59  93.83  ...  101.16  123.12  112.32  106.00  115.17  92.82   
148       101.43  94.29  ...  101.18  122.85  112.47  102.34  115.97  94.44   
149       101.17  94.83  ...  101.15  123.39  112.84   99.45  116.50  97.17   

Currency    TRY    TWD     ZAR  BRL_5yr_rolling  
145   

In [9]:
reer_data

Currency,date,BRL,CLP,CNY,COP,CZK,HUF,IDR,INR,KRW,...,PHP,PLN,RON,RUB,SGD,THB,TRY,TWD,ZAR,BRL_5yr_rolling
0,2012-04-30,162.58,120.54,86.79,144.74,100.21,112.14,107.45,93.84,92.15,...,91.12,104.30,99.18,138.88,103.94,90.75,163.32,91.41,135.04,NaN
1,2012-05-31,154.71,119.69,87.53,146.22,98.00,111.96,106.81,90.67,91.40,...,91.12,101.21,97.88,135.75,104.22,90.53,164.93,92.53,131.60,NaN
2,2012-06-30,151.64,118.77,87.40,148.25,96.79,111.99,107.06,89.71,91.84,...,92.90,101.24,97.16,131.05,104.10,90.71,165.15,92.73,130.61,NaN
3,2012-07-31,154.24,122.09,87.33,148.37,96.68,113.96,107.40,91.53,93.45,...,94.93,102.78,95.08,135.36,105.84,90.93,167.24,92.89,133.60,NaN
4,2012-08-31,153.32,124.04,86.98,145.26,97.78,116.60,106.52,91.27,93.89,...,94.33,104.26,95.85,136.11,106.56,91.02,167.51,93.00,132.25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2024-05-31,113.51,103.17,91.07,114.28,121.74,110.40,100.08,101.22,95.16,...,101.60,121.48,111.64,100.91,115.55,91.99,94.26,98.08,102.05,126.226370
146,2024-06-30,108.90,102.91,91.17,110.93,121.41,108.00,98.62,102.77,94.22,...,100.51,120.13,111.67,105.21,115.57,91.97,94.91,98.70,102.42,126.108503
147,2024-07-31,105.51,102.00,91.29,110.87,119.61,109.33,98.55,103.59,93.83,...,101.16,123.12,112.32,106.00,115.17,92.82,96.33,97.58,103.15,125.969324
148,2024-08-31,103.74,101.65,91.30,109.20,120.36,108.66,98.97,101.43,94.29,...,101.18,122.85,112.47,102.34,115.97,94.44,94.94,96.72,102.58,125.820134


In [7]:
reer_data[['CNY_5yr_rolling']]

KeyError: "None of [Index(['CNY_5yr_rolling'], dtype='object', name='Currency')] are in the [columns]"

In [9]:

def calculate_5yr_avg_reer(reer_data, currencies):
    reer_data['date'] = pd.to_datetime(reer_data['date'])
    reer_data = reer_data.sort_values('date')
    reer_data = reer_data.set_index('date')

    for currency in currencies:
        reer_data[f'{currency}_5yr_rolling'] = np.nan  # Initialize

        for i in range(len(reer_data)):
            current_date = reer_data.index[i]
            five_years_ago = current_date - pd.DateOffset(months=61) #Modified to start 61 months ago
            past_month = current_date - pd.DateOffset(months=1) #Defined past month
            past_data = reer_data.loc[five_years_ago:past_month, currency].dropna() #Selects data between 61 months ago and last month (excluded), removing NaNs

            if len(past_data) >= 60:
                reer_data.loc[current_date, f'{currency}_5yr_rolling'] = past_data.mean()

    return reer_data.reset_index(drop=False)

securities = ["BISBBRR Index", "BISBCLR Index", "BISBCNR Index",  "BISBCOP Index","BISBCZR Index", "BISBHUR Index", "BISBIDR Index", "BISBMYR Index", "BISBMXR Index",  "BISBPER Index","BISBPHR Index",  "BISBPLR Index",  "BISBROR Index",  "BISBRUR Index","BISBZAR Index", "BISBTHR Index",  "BISBTRR Index", "BISBINR Index", "BISBSGR Index", "BISBTWR Index", "BISBKRR Index"] #Use actual tickers
currencies = ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR','INR','KRW','MYR','MXN','PEN','PHP','PLN','RON','RUB','SGD','THB','TRY','TWD','ZAR']# Actual list matching ticker elements without the date column.
date = pd.to_datetime('2024-09-30') # Your target date
reer_data = calculate_5yr_avg_reer(reer_data, currencies)

# Convert the 'date' column to datetime objects if it's not already
reer_data['date'] = pd.to_datetime(reer_data['date'])

# Format the dates as 'YYYY-MM-DD' strings
reer_data['date'] = reer_data['date'].dt.strftime('%Y-%m-%d')


In [280]:
reer_data.columns 

Index(['date', 'BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'INR', 'KRW',
       'MXN', 'MYR', 'PEN', 'PHP', 'PLN', 'RON', 'RUB', 'SGD', 'THB', 'TRY',
       'TWD', 'ZAR', 'BRL_5yr_rolling', 'CLP_5yr_rolling', 'CNY_5yr_rolling',
       'COP_5yr_rolling', 'CZK_5yr_rolling', 'HUF_5yr_rolling',
       'IDR_5yr_rolling', 'INR_5yr_rolling', 'KRW_5yr_rolling',
       'MYR_5yr_rolling', 'MXN_5yr_rolling', 'PEN_5yr_rolling',
       'PHP_5yr_rolling', 'PLN_5yr_rolling', 'RON_5yr_rolling',
       'RUB_5yr_rolling', 'SGD_5yr_rolling', 'THB_5yr_rolling',
       'TRY_5yr_rolling', 'TWD_5yr_rolling', 'ZAR_5yr_rolling'],
      dtype='object', name='Currency')

In [64]:
reer_dict = {}


for date in reer_data['date'].unique():
    date_data = reer_data[reer_data['date'] == date].copy()
    date_data = date_data.drop(columns='date')

    temp_df = pd.DataFrame(index=currencies, columns=['REER', 'REER_5Y_avg'])

    for currency in currencies:
        reer_col = currency
        rolling_col = currency + "_5yr_rolling"

        if rolling_col in reer_data.columns:
            temp_df.loc[currency, 'REER'] = date_data[reer_col].iloc[0]
            temp_df.loc[currency, 'REER_5Y_avg'] = date_data[rolling_col].iloc[0]
        else:
            temp_df.loc[currency, 'REER'] = date_data[reer_col].iloc[0]
            temp_df.loc[currency, 'REER_5Y_avg'] = np.nan

    # Add Currency as a column before resetting indexFTB_3
    
    temp_df = temp_df.reset_index().rename(columns={'index': 'Currency'}) 
    reer_dict[date] = temp_df

In [65]:

# Example: Access the DataFrame for a specific date
date_key = '2024-08-31'  # Example date
if date_key in reer_dict:
    print(reer_dict[date_key])

   Currency    REER REER_5Y_avg
0       BRL  103.74  107.656721
1       CLP  101.65  102.982787
2       CNY    91.3   98.567869
3       COP   109.2    99.20377
4       CZK  120.36  111.072623
5       HUF  108.66  102.655246
6       IDR   98.97  100.642623
7       MYR   98.68   97.972131
8       MXN   124.1  114.696557
9       PEN   102.9    97.50377
10      PHP  101.18   99.824262
11      PLN  122.85   104.97541
12      RON  112.47  104.223607
13      RUB  102.34  106.682951
14      ZAR  102.58  104.140328
15      THB   94.44   96.288525
16      TRY   94.94   92.009344
17      INR  101.43  100.286885
18      SGD  115.97  105.456066
19      TWD   96.72  100.049016
20      KRW   94.29   97.865738


In [ ]:
#bquery = blp.BlpQuery().start()
LAST_UPDATE = bquery.bdh( ["BISBBRR Index"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
LAST_UPDATE_query = LAST_UPDATE.strftime('%Y%m%d')
print(LAST_UPDATE_query)

In [113]:

currencies = ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'MYR', 'MXN', 
              'PEN', 'PHP', 'PLN', 'RON', 'RUB', 'ZAR', 'THB', 'TRY', 'INR',
              'SGD', 'TWD', 'KRW']

securities = ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"]
begin = ((date - pd.DateOffset(months=62)).replace(day=1)).strftime('%Y-%m-%d') # Start_date shifted by 62months
end = date.strftime('%Y-%m-%d')
    
REER_query = blp.bdh(  # Use bdh for historical data
    securities,
    ["PX_LAST"],
    start_date=begin,
    end_date=end # Add options if needed
)
    #,options={"periodicitySelection": "MONTHLY"}
if REER_query.empty:
    return pd.DataFrame(columns=['Currency', 'REER', 'Date'])





def get_reer_data(date):
    """Retrieves REER data for a specific date."""
    begin = ((date - pd.DateOffset(months=62)).replace(day=1)).strftime('%Y-%m-%d') # Start_date shifted by 62months
    end = date.strftime('%Y-%m-%d')
    
    REER_query = blp.bdh(  # Use bdh for historical data
        securities,
        ["PX_LAST"],
        start_date=begin,
        end_date=end # Add options if needed
    )
    #,options={"periodicitySelection": "MONTHLY"}
    if REER_query.empty:
        return pd.DataFrame(columns=['Currency', 'REER', 'Date'])


    REER_query['Date'] = date
    print(REER_query)
    conditions = [
    (REER_query['security'] == 'BISBBRR Index'),
    (REER_query['security'] == 'BISBCLR Index'),
    (REER_query['security'] == 'BISBCNR Index'),
    (REER_query['security'] == 'BISBCOP Index'),
    (REER_query['security'] == 'BISBCZR Index'),
    (REER_query['security'] == 'BISBHUR Index'),
    (REER_query['security'] == 'BISBIDR Index'),
    (REER_query['security'] == 'BISBMYR Index'),
    (REER_query['security'] == 'BISBMXR Index'),
    (REER_query['security'] == 'BISBPER Index'),
    (REER_query['security'] == 'BISBPHR Index'),
    (REER_query['security'] == 'BISBPLR Index'),
    (REER_query['security'] == 'BISBROR Index'),
    (REER_query['security'] == 'BISBRUR Index'),
    (REER_query['security'] == 'BISBZAR Index'),
    (REER_query['security'] == 'BISBTHR Index'),
    (REER_query['security'] == 'BISBTRR Index'),
    (REER_query['security'] == 'BISBINR Index'),
    (REER_query['security'] == 'BISBSGR Index'),
    (REER_query['security'] == 'BISBTWR Index'),
    (REER_query['security'] == 'BISBKRR Index')
]
    REER_query.rename(columns={'PX_LAST':'REER'}, inplace=True)
    REER_query['Currency'] = np.select(conditions, currencies, default=None)   
    REER_query = REER_query[['Date', 'Currency', 'REER']] # Reorder columns
    return REER_query



# Set the date range
start_date = pd.Timestamp('2024-10-01')
end_date = pd.Timestamp('today')

# Create an empty list to store the dataframes
all_reer_data = []


# Get the latest available date for each security 
last_update_dates = pd.to_datetime(bquery.bdp(securities, ["LAST_UPDATE_DT"])["LAST_UPDATE_DT"]).to_dict()

for date in pd.date_range(start=start_date, end=end_date, freq='MS'):  # Month start frequency
    reer_df = get_reer_data(date)
    all_reer_data.append(reer_df)



# Concatenate all dataframes
final_reer_data = pd.concat(all_reer_data, ignore_index=True)
final_reer_data

           BISBBRR Index BISBCLR Index BISBCNR Index BISBCOP Index  \
                 PX_LAST       PX_LAST       PX_LAST       PX_LAST   
2019-08-31        125.64        108.87         96.12        105.69   
2019-09-30        122.76        108.40         96.25        106.51   
2019-10-31        122.86        108.03         96.95        104.80   
2019-11-30        121.11         99.85         98.34        105.56   
2019-12-31        122.93        100.18         98.02        105.89   
...                  ...           ...           ...           ...   
2024-05-31        113.51        103.17         91.07        114.28   
2024-06-30        108.90        102.91         91.17        110.93   
2024-07-31        105.51        102.00         91.29        110.87   
2024-08-31        103.74        101.65         91.28        109.20   
2024-09-30        103.71        101.29         91.33        105.84   

           BISBCZR Index BISBHUR Index BISBIDR Index BISBMYR Index  \
                 PX

KeyError: 'security'

In [22]:
# trading weights is an independent part 

from blp import blp 
weights = {'BRL': { 'EUR':19.8326891433532, 'CNY': 19.31908920559, 'USD': 18.9827326781795, 'ARS': 9.04088735496284, 'KRW': 4.00326543003275}, 
         'CLP': { 'CNY':32.4903579496702, 'USD': 15.7403934634422, 'EUR': 15.7221929477127, 'BRL': 5.30539472417201, 'KRW': 4.85057074105527},
         'CNY': { 'USD':19.6807605546305, 'EUR': 18.034148790699, 'JPY': 11.9337085872547, 'KRW': 8.77690798092872, 'TWD': 5.94072168056887},
         'COP': { 'CNY':23.7228279745679, 'USD': 22.3958427765579, 'EUR': 15.0561850379673, 'MXN': 9.47818212291651, 'BRL': 5.79804369588218},
         'CZK': { 'EUR':53.7357432005846, 'CNY': 9.32263987744015, 'PLN': 6.48343861856898, 'USD': 4.57420251197199, 'GBP': 4.46219657045916},
         'HUF': { 'EUR':51.4462139331452, 'CNY': 8.5469463341168, 'USD': 5.49254146539307, 'PLN': 4.89862553996062, 'CZK': 4.01613062014827},
         'IDR': { 'CNY':23.2668581166172, 'JPY': 12.4015844911697, 'EUR': 10.7498551165224, 'SGD': 9.67219382822659, 'USD': 9.16342505380052},
         'MYR': { 'CNY':23.7465947578572, 'USD': 13.0350647634701, 'EUR': 11.4157257777885, 'JPY': 9.45471669844536, 'SGD': 8.43320915760798},
         'MXN': { 'USD':53.7852968728671, 'CNY': 15.3570822350254, 'EUR': 8.85721047352065, 'JPY': 3.68473624173183, 'CAD': 3.65211347020733},
         'PEN': { 'CNY':27.8771478535743, 'USD': 18.0329542150098, 'EUR': 12.372696604013, 'BRL': 6.19816245225753, 'MXN': 5.04137616220383},
         'PHP': { 'CNY':20.860353198127, 'JPY': 15.316588250294, 'USD': 12.1367056047445, 'EUR': 10.5339870265713, 'KRW': 6.90886495923643},
         'PLN': { 'EUR':51.2744348107249, 'CNY': 11.8457719861224, 'GBP': 4.84680781974116, 'USD': 4.56041842863057, 'CZK': 3.76300172956997},
         'RON': { 'EUR':53.163229044445, 'CNY': 7.52963234306008, 'HUF': 4.58419055313407, 'PLN': 4.30552373422745, 'GBP': 4.06001591246192},
         'RUB': { 'EUR':36.5454149814027, 'CNY': 21.6181286031918, 'USD': 7.13934430006148, 'JPY': 4.60563918801572, 'KRW': 3.55071673784711},
         'ZAR': { 'EUR':28.0706716305097, 'CNY': 21.9557456326697, 'USD': 11.4296519508831, 'JPY': 6.34943112088636, 'GBP': 4.6456625826538},
         'THB': { 'CNY':23.5940786001256, 'JPY': 15.7028546336318, 'USD': 10.731028129059, 'EUR': 10.454098946585, 'MYR': 4.48452665287658},
         'TRY': { 'EUR':40.3579265739337, 'CNY': 14.6056217841577, 'USD': 6.35823987754735, 'GBP': 5.14480031288076, 'KRW': 3.5563885504494},
         'INR': { 'CNY':23.0813819215815, 'EUR': 18.4229102981809, 'USD': 13.9630590579442, 'KRW': 4.68345461835726, 'JPY': 4.55859195195944},
         'SGD': { 'CNY':22.4672690983772, 'EUR': 11.4224275576858, 'USD': 11.3383351154703, 'JPY': 7.95992633793645, 'MYR': 7.31117597215136},
         'TWD': { 'CNY':34.561520327848, 'JPY': 13.5244959365383, 'USD': 12.9747862420664, 'EUR': 10.1631302332482, 'KRW': 5.92150265900429},
         'KRW': { 'CNY':33.2674159298525, 'USD': 13.9679646593416, 'EUR': 12.7550195378573, 'JPY': 10.8782446505293, 'TWD': 4.03210645129297},
         'EUR': { 'CNY':18.3714673852213, 'USD': 14.5632639529848, 'GBP': 10.440394166651, 'CHF': 5.54635594069113, 'PLN': 5.50799014172826},
         'JPY': { 'CNY':31.8896059983949, 'USD': 16.4932499426125, 'EUR': 12.6796695252115, 'KRW': 5.95597200870829, 'TWD': 4.60760731009828}
}

df = pd.DataFrame.from_dict(weights, orient='index').fillna(0)
df['total_weight'] = df.apply(sum, axis =1)
df.reset_index(inplace=True)
df.rename(columns={'index':'Currency'}, inplace=True)
trading_weights = pd.melt(df,id_vars=["Currency", 'total_weight'], var_name="Trading partner", value_name="weight")
trading_weights = trading_weights[trading_weights['weight'] != 0]
print(trading_weights)

    Currency  total_weight Trading partner     weight
0        BRL     71.178664             EUR  19.832689
1        CLP     74.108910             EUR  15.722193
2        CNY     64.366248             EUR  18.034149
3        COP     76.451082             EUR  15.056185
4        CZK     78.578221             EUR  53.735743
..       ...           ...             ...        ...
307      MXN     85.336439             CAD   3.652113
334      RON     73.642592             HUF   4.584191
360      THB     64.966587             MYR   4.484527
363      SGD     60.499134             MYR   7.311176
390      EUR     54.429472             CHF   5.546356

[115 rows x 4 columns]


In [19]:
all_business_dates = pd.to_datetime(data_prices.index)
all_business_dates_str = [date.strftime('%Y-%m-%d') for date in all_business_dates]

all_business_dates_str[-50:]

['2024-09-19',
 '2024-09-20',
 '2024-09-23',
 '2024-09-24',
 '2024-09-25',
 '2024-09-26',
 '2024-09-27',
 '2024-09-30',
 '2024-10-01',
 '2024-10-02',
 '2024-10-03',
 '2024-10-04',
 '2024-10-07',
 '2024-10-08',
 '2024-10-09',
 '2024-10-10',
 '2024-10-11',
 '2024-10-14',
 '2024-10-15',
 '2024-10-16',
 '2024-10-17',
 '2024-10-18',
 '2024-10-21',
 '2024-10-22',
 '2024-10-23',
 '2024-10-24',
 '2024-10-25',
 '2024-10-28',
 '2024-10-29',
 '2024-10-30',
 '2024-10-31',
 '2024-11-01',
 '2024-11-04',
 '2024-11-05',
 '2024-11-06',
 '2024-11-07',
 '2024-11-08',
 '2024-11-11',
 '2024-11-12',
 '2024-11-13',
 '2024-11-14',
 '2024-11-15',
 '2024-11-18',
 '2024-11-19',
 '2024-11-20',
 '2024-11-21',
 '2024-11-22',
 '2024-11-25',
 '2024-11-26',
 '2024-11-27']

In [82]:
all_business_dates = pd.to_datetime(data_prices.index)


In [20]:

def complete_reer_dict(reer_dict, all_business_dates_str):
    reer_dict_completed = reer_dict.copy()  # Create a copy to avoid modifying the original


    for date_str in all_business_dates_str: #loop through all business dates (str format)

        if date_str not in reer_dict_completed:  #Check if date_str is a key in reer_dict_completed

            date = pd.to_datetime(date_str) #Convert to datetime to use pd.offsets
            # Find the most recent available date (last day of previous month)
            
            last_day_of_previous_month = (date - pd.DateOffset(months = 1) + pd.tseries.offsets.MonthEnd(0)).strftime('%Y-%m-%d')
            
            
            # Handle potential edge cases (no previous dates)
            while last_day_of_previous_month not in reer_dict_completed: #Loops to find the most recent available date.
                date_temp = pd.to_datetime(last_day_of_previous_month)
                last_day_of_previous_month = (date_temp - pd.DateOffset(months=1) + pd.tseries.offsets.MonthEnd(0)).strftime('%Y-%m-%d')

            # Copy the DataFrame
            reer_dict_completed[date_str] = reer_dict_completed[last_day_of_previous_month].copy() #Copies data from most recent key.

    return reer_dict_completed  #Returns completed dictionary
reer_dict = complete_reer_dict(reer_dict, all_business_dates_str)


In [66]:
def complete_reer_dict(reer_dict, all_business_dates_str):
    """Forward fills data within each month in the reer_dict.

    Preserves all existing keys in reer_dict. Processes dates from 2024 onwards.
    GUARANTEES no key removal.
    """
    reer_dict_completed = reer_dict.copy()
    all_business_dates = pd.to_datetime(all_business_dates_str).sort_values()
    all_business_dates_from_2024 = all_business_dates[all_business_dates >= pd.to_datetime('2024-01-01')]

    # Keep track of the latest available date *from any point in the data*
    last_available_date_overall = None
    for date_str in reer_dict:  # Pre-populate with existing dates from reer_dict if before 2024
        dt = pd.to_datetime(date_str)
        if last_available_date_overall is None or dt > last_available_date_overall:
            last_available_date_overall = dt


    for date in all_business_dates_from_2024:
        date_str = date.strftime('%Y-%m-%d')

        if date_str not in reer_dict_completed:

            previous_date = date - pd.DateOffset(days=1)
            previous_date_str = previous_date.strftime('%Y-%m-%d')



            while previous_date_str not in reer_dict_completed and previous_date >= all_business_dates[0]: # iterate from current date to earliest available date
                previous_date -= pd.DateOffset(days=1)
                previous_date_str = previous_date.strftime('%Y-%m-%d')


            if previous_date_str in reer_dict_completed:
                reer_dict_completed[date_str] = reer_dict_completed[previous_date_str].copy()

            # If no previous date exists in reer_dict_completed from 2024 onwards, check overall last_available_date_overall
            elif last_available_date_overall is not None:

                reer_dict_completed[date_str] = reer_dict[last_available_date_overall.strftime('%Y-%m-%d')].copy() # Use overall latest available date



            else:
                reer_dict_completed[date_str] = pd.DataFrame() # or preferred handling



        # Update last_available_date_overall AFTER checking and potentially filling a missing date.
        if date_str in reer_dict_completed:
            last_available_date_overall = date



    return reer_dict_completed
reer_dict2 = complete_reer_dict(reer_dict, all_business_dates_str)


In [21]:
reer_dict.keys()

dict_keys(['2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31', '2012-09-30', '2012-10-31', '2012-11-30', '2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-

In [67]:
reer_dict2.keys()

dict_keys(['2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31', '2012-09-30', '2012-10-31', '2012-11-30', '2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-

In [23]:

def complete_cpi_dict(cpi_dict, all_business_dates_str):
    """Completes the cpi_dict by forward-filling missing dates with the last available data.

    Preserves all existing keys in cpi_dict.
    """
    cpi_dict_completed = cpi_dict.copy()
    all_business_dates = pd.to_datetime(all_business_dates_str).sort_values()

    last_available_date_str = None  # Keep track of the last date with available data
    last_available_value = None

    for date in all_business_dates:  # Iterate through all business dates
        date_str = date.strftime('%Y-%m-%d')

        if date_str in cpi_dict_completed: #Update the values for existing dates
            last_available_date_str = date_str
            last_available_value = cpi_dict_completed[date_str]
        elif last_available_value is not None : #If it is a missing date, fill with last available value
            cpi_dict_completed[date_str] = last_available_value

    return cpi_dict_completed

cpi_dict_2 = complete_cpi_dict(cpi_dict, all_business_dates_str)


In [89]:

def complete_reer_dict(reer_dict, all_business_dates_str):
    reer_dict_completed = reer_dict.copy()

    all_business_dates = pd.to_datetime(all_business_dates_str)

    for date in all_business_dates:
        date_str = date.strftime('%Y-%m-%d')
        
        if date_str not in reer_dict_completed:
            
            previous_date = date - pd.DateOffset(days=1) # Start from previous day
            previous_date_str = previous_date.strftime('%Y-%m-%d')


            while previous_date_str not in reer_dict_completed and previous_date >= all_business_dates[0]: #loop until an existing date is found or until first date
                previous_date -= pd.DateOffset(days=1)
                previous_date_str = previous_date.strftime('%Y-%m-%d')

            if previous_date_str in reer_dict_completed: # if a suitable previous date was found

                reer_dict_completed[date_str] = reer_dict_completed[previous_date_str].copy()

            #No Suitable Date found, use first value
            elif all_business_dates[0].strftime('%Y-%m-%d') in reer_dict_completed:

                reer_dict_completed[date_str] = reer_dict_completed[all_business_dates[0].strftime('%Y-%m-%d')].copy() # Use earliest date
            else:
                raise KeyError("No suitable previous date found for completing the dictionary")



    return reer_dict_completed
reer_dict = complete_reer_dict(reer_dict, all_business_dates_str)


KeyError: 'No suitable previous date found for completing the dictionary'

In [138]:
cpi_dict_2 = complete_reer_dict(cpi_dict, all_business_dates_str)
#cpi_dict = cpi_dict_2.copy()

In [146]:
cpi_dict2 = complete_reer_dict(cpi_dict, all_business_dates_str)

In [147]:
cpi_dict2.keys()

dict_keys(['2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30', '2021-12-31', '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-

# reer_dict2 and cpi_dict2

In [30]:
#ADJUSTED SPOT
today = pd.Timestamp('today')
last_day_of_previous_month = (today - pd.DateOffset(months=1) + pd.tseries.offsets.MonthEnd(0)).strftime('%Y%m%d')
LAST_UPDATE_query = last_day_of_previous_month
print(LAST_UPDATE_query)
today = pd.Timestamp('today')
endofmonth = ((today - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1))


20241031


In [148]:

def print_last_update(reer_dict):
    for date_str in all_business_dates_str: #Iterate through reer_dict's keys

        date = pd.to_datetime(date_str)
        last_day_of_previous_month = (date - pd.DateOffset(months=1) + pd.tseries.offsets.MonthEnd(0)).strftime('%Y%m%d') #Most robust way to calculate last day of previous month
        print(f"For date: {date_str}, LAST_UPDATE_query: {last_day_of_previous_month}")


# Example usage (assuming you have reer_dict created):
print_last_update(reer_dict)

For date: 2019-11-26, LAST_UPDATE_query: 20191031
For date: 2019-11-27, LAST_UPDATE_query: 20191031
For date: 2019-11-28, LAST_UPDATE_query: 20191031
For date: 2019-11-29, LAST_UPDATE_query: 20191031
For date: 2019-12-02, LAST_UPDATE_query: 20191130
For date: 2019-12-03, LAST_UPDATE_query: 20191130
For date: 2019-12-04, LAST_UPDATE_query: 20191130
For date: 2019-12-05, LAST_UPDATE_query: 20191130
For date: 2019-12-06, LAST_UPDATE_query: 20191130
For date: 2019-12-09, LAST_UPDATE_query: 20191130
For date: 2019-12-10, LAST_UPDATE_query: 20191130
For date: 2019-12-11, LAST_UPDATE_query: 20191130
For date: 2019-12-12, LAST_UPDATE_query: 20191130
For date: 2019-12-13, LAST_UPDATE_query: 20191130
For date: 2019-12-16, LAST_UPDATE_query: 20191130
For date: 2019-12-17, LAST_UPDATE_query: 20191130
For date: 2019-12-18, LAST_UPDATE_query: 20191130
For date: 2019-12-19, LAST_UPDATE_query: 20191130
For date: 2019-12-20, LAST_UPDATE_query: 20191130
For date: 2019-12-23, LAST_UPDATE_query: 20191130


In [ ]:
spot_tday =  bquery.bdp( [f"{currency}{trading_partner} Curncy"],["PX_LAST"]).loc[0,'PX_LAST'] # adapt with a bdh for the spot of the day with dataframe spot historic 


In [38]:
## OK line works 
LAST_UPDATE_query = '20241031'
currency = 'BRL'
trading_partner = 'USD'
date_current = '20241126'
spot_tday = bquery.bdh([f"{currency}{trading_partner} Curncy"],["PX_LAST"], start_date= date_current, end_date= date_current, options={"adjustmentSplit": True}).loc[0,'PX_LAST']
spot_last = bquery.bdh([f"{currency}{trading_partner} Curncy"],["PX_LAST"], start_date= LAST_UPDATE_query, end_date= LAST_UPDATE_query, options={"adjustmentSplit": True}).loc[0,'PX_LAST']

In [26]:
spot_tday

0.1721

In [39]:
def get_adjusted_spot(row, date_current):  # Changed df to row
    """Calculates the adjusted spot."""
    currency = row['Currency']  # Access individual values, not Series
    trading_partner = row['Trading partner'] # Access individual values, not Series
    weight = row['weight']
    total_weight = row['total_weight']

    security_ticker = f"{currency}{trading_partner} Curncy"  # Correct ticker format


    try:  # Important: Handle potential Bloomberg errors!
        spot_tday = bquery.bdh([security_ticker], ["PX_LAST"], start_date=date_current, end_date=date_current, options={"adjustmentSplit": True}).loc[0, 'PX_LAST']
        spot_last = bquery.bdh([security_ticker], ["PX_LAST"], start_date=LAST_UPDATE_query, end_date=LAST_UPDATE_query, options={"adjustmentSplit": True}).loc[0, 'PX_LAST']


        return ((spot_tday / spot_last) - 1) * weight / total_weight

    except Exception as e:  # Handle any Bloomberg API or data errors
        print(f"Error retrieving data for {security_ticker}: {e}")
        return np.nan  # Return NaN for missing or incorrect data



trading_weights['spot_adj'] = trading_weights.apply(get_adjusted_spot, date_current='20241126', axis=1) #date as string

Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0


In [43]:
trading_weights

,Currency,total_weight,Trading partner,weight,spot_adj
0,BRL,71.178664,EUR,19.832689,0.009477
1,CLP,74.108910,EUR,15.722193,0.004216
2,CNY,64.366248,EUR,18.034149,0.005615
3,COP,76.451082,EUR,15.056185,0.008538
4,CZK,78.578221,EUR,53.735743,0.000000
...,...,...,...,...,...
307,MXN,85.336439,CAD,3.652113,-0.000984
334,RON,73.642592,HUF,4.584191,0.000300
360,THB,64.966587,MYR,4.484527,-0.000687
363,SGD,60.499134,MYR,7.311176,-0.000051


In [155]:
def get_5Y_average(df):
    security = df['security']
    LAST_UPDATE = bquery.bdp( [f"{security}"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
    print(LAST_UPDATE)
    start_date = (LAST_UPDATE - relativedelta(months=62)).strftime('%Y-%m-%d')
    end_date = LAST_UPDATE.strftime('%Y-%m-%d')
    query_text = f"get(avg(px_last(dates=range({start_date}, {end_date}), Frq=M, fill=PREV))) for(['{security}'])"
    BQL_query = bquery.bql(expression=f"{query_text}")
    return BQL_query.loc[1,'value']

REER_query['REER_5Y_avg'] = REER_query.apply(get_5Y_average, axis=1)
REER_query.reset_index(inplace=True)
REER_query=REER_query[['Currency', 'REER', 'REER_5Y_avg']]

2024-10-31 00:00:00


KeyError: 1

In [54]:
reer_dict.keys()

dict_keys(['2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31', '2012-09-30', '2012-10-31', '2012-11-30', '2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-

In [127]:
def calculate_reer_discount(reer_dict2, trading_weights, cpi_dict_2, all_business_dates_str ):
    start_date_str = "2024-11-23"

# Create a filtered list of date strings
    filtered_dates = [date_str for date_str in all_business_dates_str if pd.to_datetime(date_str) > pd.to_datetime(start_date_str)]


    bquery = blp.BlpQuery().start() #Bloomberg query object
    for date_str in filtered_dates: #Iterate through all business dates (string format)
        
        if date_str in reer_dict2:
            print(date_str)
            #CPI_query = cpi_dict_2[date_str]
            #print(CPI_query)
            date = pd.to_datetime(date_str)
            last_day_of_previous_month = (date - pd.DateOffset(months=1) + pd.tseries.offsets.MonthEnd(0)).strftime('%Y%m%d')
            LAST_UPDATE_query = last_day_of_previous_month
            print(LAST_UPDATE_query)
            endofmonth = ((date- pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1))
            today = pd.Timestamp('today')

            date_query = date.strftime('%Y%m%d')

            # ADJUSTED SPOT (No changes needed, uses date and LAST_UPDATE_query)
            trading_weights['spot_adj'] = trading_weights.apply(get_adjusted_spot, date_current=date_query, axis=1)
              #Apply  get_adjusted spot with correct date variables

            # ADJUSTED INFLATION (Use date_str to access cpi_dict)
            try: #Try/except to handle KeyError if a date is missing
                # cpi_dict_2 is completed 
                print(type(date_str))
                CPI_query = cpi_dict_2[date_str]
            except KeyError:
                print(f"CPI data not found for {date_str}. Skipping REER discount calculation.")
                continue  # Go to the next date


            add_ccycpi = pd.merge(trading_weights, CPI_query, on='Currency')
            add_ccycpi.rename(columns={'CPI':'CPI_ccy'}, inplace=True)
            add_tpcpi = pd.merge(add_ccycpi, CPI_query, left_on='Trading partner', right_on='Currency').drop(['Currency_y'], axis = 1)
            add_tpcpi.rename(columns={'Currency_x':'Currency', 'CPI':'CPI_tp'}, inplace=True)
            add_tpcpi['infla_adj']=((add_tpcpi['CPI_ccy']-add_tpcpi['CPI_tp'])/100)*(add_tpcpi['weight']/add_tpcpi['total_weight'])
            add_tpcpi=add_tpcpi[['Currency','spot_adj','infla_adj']]

            # REER DISCOUNT PROXY (No changes, uses today, LAST_UPDATE, endofmonth)
            REER_proxy = add_tpcpi.groupby('Currency').sum().reset_index()
            REER_proxy['proxy'] = (1 + REER_proxy['infla_adj'])**((today - pd.to_datetime(LAST_UPDATE_query)).days/(endofmonth - pd.to_datetime(LAST_UPDATE_query)).days) * (1+REER_proxy['spot_adj']) #Corrected: Convert LAST_UPDATE_query to datetime
                    
            reer_dict2[date_str] = pd.merge(reer_dict2[date_str], REER_proxy, on='Currency')
            reer_dict2[date_str]['REER_Discount']=(reer_dict2[date_str]['REER']*reer_dict2[date_str]['proxy']/reer_dict2[date_str]['REER_5Y_avg'])-1
            reer_dict2[date_str].sort_values(by='Currency', ascending=True, inplace=True)
            

            # Merge and update reer_dict (use date_str as key)
            #reer_dict2[date_str] = pd.merge(reer_dict2[date_str], REER_proxy[['Currency', 'proxy']], on='Currency', how='left')

            #reer_dict2[date_str] = reer_dict2[date_str].reset_index().rename(columns={'index':'Currency'})
            #reer_dict2[date_str] = reer_dict2[date_str].set_index('Currency')



    bquery.stop() #Stop Bloomberg query session
    return reer_dict2

In [89]:
cpi_dict_2['2024-11-27']

,Currency,CPI
0,BRL,-0.02
1,CLP,0.3
2,CNY,0.4
3,COP,0.0
4,CZK,0.3
5,HUF,0.0
6,IDR,-0.03
7,MYR,0.1
8,MXN,0.01
9,PEN,0.31


In [80]:
reer_dict2['2024-11-25']

,Currency,REER,REER_5Y_avg
0,BRL,103.66,106.868361
1,CLP,102.22,102.706557
2,CNY,92.01,98.39377
3,COP,104.64,99.188197
4,CZK,119.03,111.737541
5,HUF,106.12,102.799016
6,IDR,99.73,100.533115
7,MYR,101.11,97.83918
8,MXN,121.26,115.177541
9,PEN,102.27,97.508525


In [128]:
reer_dict_test = reer_dict2.copy()

In [ ]:
reer_dict_test['2024']

In [129]:
reer_dict_test = calculate_reer_discount(reer_dict_test, trading_weights, cpi_dict_2, all_business_dates_str )

2024-11-25
20241031
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>
2024-11-26
20241031
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>
2024-11-27
20241031
Error retrieving data for IDRUSD Curncy: 0
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>


In [143]:
reer_dict_test['2024-11-26']

,Currency,REER,REER_5Y_avg,spot_adj,infla_adj,proxy,REER_Discount
0,BRL,103.66,106.868361,0.014396,-0.007391,1.007646,-0.022605
1,CLP,102.22,102.706557,-0.000432,0.000574,1.000084,-0.004654
2,CNY,92.01,98.39377,-0.002641,0.001572,0.99877,-0.06603
3,COP,104.64,99.188197,0.022501,-0.001728,1.020911,0.077024
4,CZK,119.03,111.737541,-0.005325,0.001531,0.996045,0.061051
5,HUF,106.12,102.799016,-0.011076,-0.001453,0.987631,0.019537
6,IDR,99.73,100.533115,0.002156,-0.004019,0.99853,-0.009447
17,INR,103.17,100.301967,0.014954,0.036182,1.047946,0.077911
20,KRW,93.64,97.69918,0.000142,0.000925,1.000975,-0.040613
8,MXN,121.26,115.177541,-0.022293,-0.001484,0.976401,0.027964


In [13]:
#Foward points (ask) : 3M and 1M
bquery = blp.BlpQuery().start()

bquery = blp.BlpQuery().start()
fpa3m_query = bquery.bdh(
        ["BCN3M BGN Curncy","CHN3M BGN Curncy","CNH3M CMPN Curncy","CLN3M BGN Curncy","CZK3M BGN Curncy","HUF3M BGN Curncy","IHN3M CMPN Curncy","MRN3M CMPN Curncy","MXN3M BGN Curncy","PSN3M BGN Curncy","PPN3M CMPN Curncy","PLN3M BGN Curncy","RON3M BGN Curncy","RUB3M BGN Curncy","ZAR3M BGN Curncy","THB3M BGN Curncy","TRY3M BGN Curncy","IRN3M CMPN Curncy","SGD3M BGN Curncy","NTN3M BGN Curncy","KWN3M CMPN Curncy"],
        ["PX_MID", "FWD_SCALE"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

fpa3m_query['Cntry_code'] = fpa3m_query['security'].str[:3]
fpa3m_query.rename(columns={'PX_MID':'FPA3M_MID'}, inplace=True)

fpa1m_query = bquery.bdp(
        ["BCN1M BGN Curncy","CHN1M BGN Curncy","CNH1M CMPN Curncy","CLN1M BGN Curncy","CZK1M BGN Curncy","HUF1M BGN Curncy","IHN1M CMPN Curncy","MRN1M CMPN Curncy","MXN1M BGN Curncy","PSN1M BGN Curncy","PPN1M CMPN Curncy","PLN1M BGN Curncy","RON1M BGN Curncy","RUB1M BGN Curncy","ZAR1M BGN Curncy","THB1M BGN Curncy","TRY1M BGN Curncy","IRN1M CMPN Curncy","SGD1M BGN Curncy","NTN1M BGN Curncy","KWN1M CMPN Curncy"],
        ["PX_MID"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

fpa1m_query['Cntry_code'] = fpa1m_query['security'].str[:3]
fpa1m_query.rename(columns={'PX_MID':'FPA1M_MID'}, inplace=True)

fpa_query = pd.merge(fpa3m_query, fpa1m_query, on='Cntry_code')

conditions = [
    (fpa_query['Cntry_code'] == "BCN"),
    (fpa_query['Cntry_code'] == "CHN"),
    (fpa_query['Cntry_code'] == "CNH"),
    (fpa_query['Cntry_code'] == "CLN"),
    (fpa_query['Cntry_code'] == "CZK"),
    (fpa_query['Cntry_code'] == "HUF"),
    (fpa_query['Cntry_code'] == "IHN"),
    (fpa_query['Cntry_code'] == "MRN"),
    (fpa_query['Cntry_code'] == "MXN"),
    (fpa_query['Cntry_code'] == "PSN"),
    (fpa_query['Cntry_code'] == "PPN"),
    (fpa_query['Cntry_code'] == "PLN"),
    (fpa_query['Cntry_code'] == "RON"),
    (fpa_query['Cntry_code'] == "RUB"),
    (fpa_query['Cntry_code'] == "ZAR"),
    (fpa_query['Cntry_code'] == "THB"),
    (fpa_query['Cntry_code'] == "TRY"),
    (fpa_query['Cntry_code'] == "IRN"),
    (fpa_query['Cntry_code'] == "SGD"),
    (fpa_query['Cntry_code'] == "NTN"),
    (fpa_query['Cntry_code'] == "KWN")
]

fpa_query['Currency'] = np.select(conditions, currencies, default=None)
fpa_query.sort_values(by='Currency', ascending=True, inplace=True)
fpa_query.reset_index(inplace=True)
fpa_query['FPA3M_MID']=fpa_query['FPA3M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query['FPA1M_MID']=fpa_query['FPA1M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query= fpa_query[['Currency', 'FWD_SCALE','FPA3M_MID','FPA1M_MID']]
fpa_query

TypeError: Response for TRY3M BGN Curncy contains fieldException {'fieldId': 'FWD_SCALE', 'errorInfo': {'errorInfo': {'source': '53361:rsfhdsvc3', 'code': 1, 'category': 'BAD_FLD', 'message': 'Not valid historical field', 'subcategory': 'NOT_APPLICABLE_TO_HIST_DATA'}}}

In [8]:
from xbbg import blp 
from blp import blp 
bquery = blp.BlpQuery().start() # Use xbbg.blp.BlpQuery

today = date.today()
end_date = today.strftime("%Y%m%d")
strt_date = "20221230"

fpa3m_query = bquery.bdp(
    [  # Verify these tickers!
        "BCN3M BGN Curncy", "CHN3M BGN Curncy", "CNH3M CMPN Curncy",
        "CLN3M BGN Curncy", "CZK3M BGN Curncy", "HUF3M BGN Curncy",
        "IHN3M CMPN Curncy", "MRN3M CMPN Curncy", "MXN3M BGN Curncy",
        "PSN3M BGN Curncy", "PPN3M CMPN Curncy", "PLN3M BGN Curncy",
        "RON3M BGN Curncy", "RUB3M BGN Curncy", "ZAR3M BGN Curncy",
        "THB3M BGN Curncy", "TRY3M BGN Curncy", "IRN3M CMPN Curncy",
        "SGD3M BGN Curncy", "NTN3M BGN Curncy", "KWN3M CMPN Curncy"
    ],
    ["PX_MID","FWD_SCALE"],
    options={"adjustmentSplit": True, "CDR": "5D"}  # Corrected options syntax
)

fpa3m_query 

# ... (Rest of your code to process the 'spot_query' result) ...

bquery.stop() # Important to stop the Bloomberg session when finished.


NotFoundException: Attempt to access unknown sub-element 'adjustmentSplit' on element 'ReferenceDataRequest' (0x0006000d)

In [5]:
#Foward points (ask) : 3M and 1M
bquery = blp.BlpQuery().start()
fpa3m_query = bquery.bdp(
        ["BCN3M BGN Curncy","CHN3M BGN Curncy","CNH3M CMPN Curncy","CLN3M BGN Curncy","CZK3M BGN Curncy","HUF3M BGN Curncy","IHN3M CMPN Curncy","MRN3M CMPN Curncy","MXN3M BGN Curncy","PSN3M BGN Curncy","PPN3M CMPN Curncy","PLN3M BGN Curncy","RON3M BGN Curncy","RUB3M BGN Curncy","ZAR3M BGN Curncy","THB3M BGN Curncy","TRY3M BGN Curncy","IRN3M CMPN Curncy","SGD3M BGN Curncy","NTN3M BGN Curncy","KWN3M CMPN Curncy"],
        ["PX_MID", "FWD_SCALE"])

fpa3m_query['Cntry_code'] = fpa3m_query['security'].str[:3]
fpa3m_query.rename(columns={'PX_MID':'FPA3M_MID'}, inplace=True)

fpa1m_query = bquery.bdp(
        ["BCN1M BGN Curncy","CHN1M BGN Curncy","CNH1M CMPN Curncy","CLN1M BGN Curncy","CZK1M BGN Curncy","HUF1M BGN Curncy","IHN1M CMPN Curncy","MRN1M CMPN Curncy","MXN1M BGN Curncy","PSN1M BGN Curncy","PPN1M CMPN Curncy","PLN1M BGN Curncy","RON1M BGN Curncy","RUB1M BGN Curncy","ZAR1M BGN Curncy","THB1M BGN Curncy","TRY1M BGN Curncy","IRN1M CMPN Curncy","SGD1M BGN Curncy","NTN1M BGN Curncy","KWN1M CMPN Curncy"],
        ["PX_MID"])

fpa1m_query['Cntry_code'] = fpa1m_query['security'].str[:3]
fpa1m_query.rename(columns={'PX_MID':'FPA1M_MID'}, inplace=True)

fpa_query = pd.merge(fpa3m_query, fpa1m_query, on='Cntry_code')

conditions = [
    (fpa_query['Cntry_code'] == "BCN"),
    (fpa_query['Cntry_code'] == "CHN"),
    (fpa_query['Cntry_code'] == "CNH"),
    (fpa_query['Cntry_code'] == "CLN"),
    (fpa_query['Cntry_code'] == "CZK"),
    (fpa_query['Cntry_code'] == "HUF"),
    (fpa_query['Cntry_code'] == "IHN"),
    (fpa_query['Cntry_code'] == "MRN"),
    (fpa_query['Cntry_code'] == "MXN"),
    (fpa_query['Cntry_code'] == "PSN"),
    (fpa_query['Cntry_code'] == "PPN"),
    (fpa_query['Cntry_code'] == "PLN"),
    (fpa_query['Cntry_code'] == "RON"),
    (fpa_query['Cntry_code'] == "RUB"),
    (fpa_query['Cntry_code'] == "ZAR"),
    (fpa_query['Cntry_code'] == "THB"),
    (fpa_query['Cntry_code'] == "TRY"),
    (fpa_query['Cntry_code'] == "IRN"),
    (fpa_query['Cntry_code'] == "SGD"),
    (fpa_query['Cntry_code'] == "NTN"),
    (fpa_query['Cntry_code'] == "KWN")
]

fpa_query['Currency'] = np.select(conditions, currencies, default=None)
fpa_query.sort_values(by='Currency', ascending=True, inplace=True)
fpa_query.reset_index(inplace=True)
fpa_query['FPA3M_MID']=fpa_query['FPA3M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query['FPA1M_MID']=fpa_query['FPA1M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query= fpa_query[['Currency', 'FWD_SCALE','FPA3M_MID','FPA1M_MID']]
fpa_query

,Currency,FWD_SCALE,FPA3M_MID,FPA1M_MID
0,BRL,4,0.073002,0.023680
1,CLP,0,0.950000,0.300000
2,CNY,4,-0.026666,-0.005135
3,COP,0,53.125000,20.500000
4,CZK,3,-0.056370,-0.014125
5,HUF,2,1.692750,0.555500
6,IDR,0,75.000000,26.500000
7,INR,2,0.361900,0.106700
8,KRW,0,-4.975000,-1.000000
9,MXN,4,0.319350,0.108124


In [18]:
fpa_query[['FWD_SCALE', 'Currency']]

,FWD_SCALE,Currency
0,4,BRL
1,0,CLP
2,4,CNY
3,0,COP
4,3,CZK
5,2,HUF
6,0,IDR
7,2,INR
8,0,KRW
9,4,MXN


In [30]:
end_date

'20241108'

In [6]:
fwd_scale = fpa_query[['FWD_SCALE', 'Currency']]
fwd_scale_dict = dict(zip(fwd_scale['Currency'], fwd_scale['FWD_SCALE'])) # Dictionary for efficient lookup

bquery = blp.BlpQuery().start()
fpa3m_query = bquery.bdh(
        ["BCN3M BGN Curncy","CHN3M BGN Curncy","CNH3M CMPN Curncy","CLN3M BGN Curncy","CZK3M BGN Curncy","HUF3M BGN Curncy","IHN3M CMPN Curncy","MRN3M CMPN Curncy","MXN3M BGN Curncy","PSN3M BGN Curncy","PPN3M CMPN Curncy","PLN3M BGN Curncy","RON3M BGN Curncy","RUB3M BGN Curncy","ZAR3M BGN Curncy","THB3M BGN Curncy","TRY3M BGN Curncy","IRN3M CMPN Curncy","SGD3M BGN Curncy","NTN3M BGN Curncy","KWN3M CMPN Curncy"],
        ["PX_MID"],
        start_date=strt_date,
    end_date=end_date,
    options={"adjustmentSplit": True, "CDR": "5D"} )

fpa3m_query['Cntry_code'] = fpa3m_query['security'].str[:3]
fpa3m_query.rename(columns={'PX_MID':'FPA3M_MID'}, inplace=True)

fpa1m_query = bquery.bdh(
        ["BCN1M BGN Curncy","CHN1M BGN Curncy","CNH1M CMPN Curncy","CLN1M BGN Curncy","CZK1M BGN Curncy","HUF1M BGN Curncy","IHN1M CMPN Curncy","MRN1M CMPN Curncy","MXN1M BGN Curncy","PSN1M BGN Curncy","PPN1M CMPN Curncy","PLN1M BGN Curncy","RON1M BGN Curncy","RUB1M BGN Curncy","ZAR1M BGN Curncy","THB1M BGN Curncy","TRY1M BGN Curncy","IRN1M CMPN Curncy","SGD1M BGN Curncy","NTN1M BGN Curncy","KWN1M CMPN Curncy"],
        ["PX_MID"],
        start_date=strt_date,
    end_date=end_date,
    options={"adjustmentSplit": True, "CDR": "5D"} )

fpa1m_query['Cntry_code'] = fpa1m_query['security'].str[:3]
fpa1m_query.rename(columns={'PX_MID':'FPA1M_MID'}, inplace=True)

fpa_query = pd.merge(fpa3m_query, fpa1m_query, on='Cntry_code')

conditions = [
    (fpa_query['Cntry_code'] == "BCN"),
    (fpa_query['Cntry_code'] == "CHN"),
    (fpa_query['Cntry_code'] == "CNH"),
    (fpa_query['Cntry_code'] == "CLN"),
    (fpa_query['Cntry_code'] == "CZK"),
    (fpa_query['Cntry_code'] == "HUF"),
    (fpa_query['Cntry_code'] == "IHN"),
    (fpa_query['Cntry_code'] == "MRN"),
    (fpa_query['Cntry_code'] == "MXN"),
    (fpa_query['Cntry_code'] == "PSN"),
    (fpa_query['Cntry_code'] == "PPN"),
    (fpa_query['Cntry_code'] == "PLN"),
    (fpa_query['Cntry_code'] == "RON"),
    (fpa_query['Cntry_code'] == "RUB"),
    (fpa_query['Cntry_code'] == "ZAR"),
    (fpa_query['Cntry_code'] == "THB"),
    (fpa_query['Cntry_code'] == "TRY"),
    (fpa_query['Cntry_code'] == "IRN"),
    (fpa_query['Cntry_code'] == "SGD"),
    (fpa_query['Cntry_code'] == "NTN"),
    (fpa_query['Cntry_code'] == "KWN")
]

fpa_query['Currency'] = np.select(conditions, currencies, default=None)
fpa_query.sort_values(by='Currency', ascending=True, inplace=True)
fpa_query.reset_index(inplace=True)
fpa_query['FWD_SCALE'] = fpa_query['Currency'].map(fwd_scale_dict) 
fpa_query['FPA3M_MID'] = fpa_query['FPA3M_MID'] / (10**fpa_query['FWD_SCALE'])
fpa_query['FPA1M_MID'] = fpa_query['FPA1M_MID'] / (10**fpa_query['FWD_SCALE'])

fpa_query= fpa_query[['Currency', 'FWD_SCALE','FPA3M_MID','FPA1M_MID']]
fpa_query

,Currency,FWD_SCALE,FPA3M_MID,FPA1M_MID
0,BRL,4,0.067656,0.018530
1,BRL,4,0.064277,0.020470
2,BRL,4,0.064277,0.020200
3,BRL,4,0.064277,0.019860
4,BRL,4,0.064277,0.017391
...,...,...,...,...
3019,ZAR,4,0.143290,0.048090
3020,ZAR,4,0.143290,0.051070
3021,ZAR,4,0.143290,0.052530
3022,ZAR,4,0.146820,0.048090


In [23]:
fpa3m_query

,date,security,FPA3M_MID,Cntry_code
0,2022-12-30,TRY3M BGN Curncy,13217.075,TRY
1,2022-12-30,THB3M BGN Curncy,-43.000,THB
2,2022-12-30,IRN3M CMPN Curncy,59.000,IRN
3,2022-12-30,CZK3M BGN Curncy,110.000,CZK
4,2022-12-30,MRN3M CMPN Curncy,-261.993,MRN
...,...,...,...,...
10201,2024-11-08,IRN3M CMPN Curncy,35.050,IRN
10202,2024-11-08,RUB3M BGN Curncy,32113.500,RUB
10203,2024-11-08,TRY3M BGN Curncy,32241.387,TRY
10204,2024-11-08,MXN3M BGN Curncy,3033.500,MXN


In [39]:
bquery = blp.BlpQuery().start()
IMPVOL_query = bquery.bdh(
        ["USDBRLV3M Curncy","USDCLPV3M Curncy","USDCNYV3M Curncy","USDCOPV3M Curncy","USDCZKV3M Curncy","USDHUFV3M Curncy","USDIDRV3M Curncy","USDMYRV3M Curncy","USDMXNV3M Curncy","USDPENV3M Curncy","USDPHPV3M Curncy","USDPLNV3M Curncy","USDRONV3M Curncy","USDRUBV3M Curncy","USDZARV3M Curncy","USDTHBV3M Curncy","USDTRYV3M Curncy","USDINRV3M Curncy","USDSGDV3M Curncy","USDTWDV3M Curncy","USDKRWV3M Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"}) #

IMPVOL_query.rename(columns={'date':'DATE', 'PX_LAST':'3M Implied Vol'}, inplace=True)

conditions = [
    (IMPVOL_query['security'] == 'USDBRLV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCLPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCNYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCOPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCZKV3M Curncy'),
    (IMPVOL_query['security'] == 'USDHUFV3M Curncy'),
    (IMPVOL_query['security'] == 'USDIDRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMYRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMXNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPENV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPHPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPLNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRONV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRUBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDZARV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTHBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTRYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDINRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDSGDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTWDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDKRWV3M Curncy'),
]

IMPVOL_query['Currency'] = np.select(conditions, currencies, default=None)
IMPVOL_query.drop(columns="security", inplace=True)
IMPVOL_query.sort_values(by='DATE', ascending=False, inplace=True)
IMPVOL_query=IMPVOL_query[['DATE','Currency','3M Implied Vol']]
IMPVOL_query

,DATE,Currency,3M Implied Vol
4766,2024-11-12,MYR,6.3875
4756,2024-11-12,HUF,11.7600
4746,2024-11-12,RON,7.5850
4747,2024-11-12,PEN,4.3875
4748,2024-11-12,TWD,6.8500
...,...,...,...
17,2024-01-01,PHP,6.5325
18,2024-01-01,CLP,15.7600
19,2024-01-01,PEN,7.4225
20,2024-01-01,CNY,4.5000


In [150]:
strt_date

'20241030'

In [174]:
from blp import blp 
end_date_str = pd.Timestamp(end_date).strftime('%Y%m%d') 
strt_date = '2018-01-01'
currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]

bquery = blp.BlpQuery().start()

GDP_query = bquery.bdh(
        ["WGDPBRAZ Index","WGDPCHIL Index","WGDPCHIN Index","WGDPCOLO Index","WGDPCZEC Index","WGDPHUNG Index","WGDPINDO Index","WGDPMALY Index","WGDPMEXI Index","WGDPPERU Index","WGDPPHIL Index","WGDPPOLA Index","WGDPROMA Index","WGDPRUSS Index","WGDPSOUT Index","WGDPTHAI Index","WGDPTURK Index","WGDPINDI Index","WGDPSING Index","TAGDPARE Index","WGDPKORE Index"],
        ["PX_LAST"],
        start_date= '20210101', 
        end_date= end_date_str, 
        options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"YEARLY", 'nonTradingDayFillMethod':'NIL_VALUE'})

GDP_query.rename(columns={'PX_LAST':'GDP'}, inplace=True)

conditions = [
    (GDP_query['security'] == 'WGDPBRAZ Index'),
    (GDP_query['security'] == 'WGDPCHIL Index'),
    (GDP_query['security'] == 'WGDPCHIN Index'),
    (GDP_query['security'] == 'WGDPCOLO Index'),
    (GDP_query['security'] == 'WGDPCZEC Index'),
    (GDP_query['security'] == 'WGDPHUNG Index'),
    (GDP_query['security'] == 'WGDPINDO Index'),
    (GDP_query['security'] == 'WGDPMALY Index'),
    (GDP_query['security'] == 'WGDPMEXI Index'),
    (GDP_query['security'] == 'WGDPPERU Index'),
    (GDP_query['security'] == 'WGDPPHIL Index'),
    (GDP_query['security'] == 'WGDPPOLA Index'),
    (GDP_query['security'] == 'WGDPROMA Index'),
    (GDP_query['security'] == 'WGDPRUSS Index'),
    (GDP_query['security'] == 'WGDPSOUT Index'),
    (GDP_query['security'] == 'WGDPTHAI Index'),
    (GDP_query['security'] == 'WGDPTURK Index'),
    (GDP_query['security'] == 'WGDPINDI Index'),
    (GDP_query['security'] == 'WGDPSING Index'),
    (GDP_query['security'] == 'TAGDPARE Index'),
    (GDP_query['security'] == 'WGDPKORE Index')
]

GDP_query['Currency'] = np.select(conditions, currencies, default=None)
GDP_query.sort_values(by='Currency', ascending=True, inplace=True)
GDP_query.reset_index(inplace=True)
GDP_query=GDP_query[['Currency','GDP']]
GDP_query

,Currency,GDP
0,BRL,2173.666
1,BRL,1670.647
2,BRL,1951.924
3,CLP,335.533
4,CLP,315.515
...,...,...
58,TWD,21178.950
59,TWD,22005.120
60,ZAR,420.118
61,ZAR,377.782


In [223]:
securities = ["WGDPBRAZ Index","WGDPCHIL Index","WGDPCHIN Index","WGDPCOLO Index","WGDPCZEC Index","WGDPHUNG Index","WGDPINDO Index","WGDPMALY Index","WGDPMEXI Index","WGDPPERU Index","WGDPPHIL Index","WGDPPOLA Index","WGDPROMA Index","WGDPRUSS Index","WGDPSOUT Index","WGDPTHAI Index","WGDPTURK Index","WGDPINDI Index","WGDPSING Index","TAGDPARE Index","WGDPKORE Index"]
fields = ["PX_LAST"]
start_date= '20180101'
end_date= end_date_str
options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"YEARLY", 'nonTradingDayFillMethod':'NIL_VALUE'}

GDP_data = [] # list to store individual dataframes

for security in securities:
    try:
        df = bquery.bdh([security], fields, start_date=start_date, end_date=end_date_str, options=options)

        if not df.empty:
            df['Currency'] = currencies[securities.index(security)]  # Add currency using index
            GDP_data.append(df) # Append DataFrame
        else:
            print(f"No data retrieved for {security}")

    except Exception as e:
        print(f"Error retrieving data for {security}: {e}")

In [176]:
GDP_data[1]

,date,security,PX_LAST,Currency
0,2018-12-31,WGDPCHIL Index,295.858,CLP
1,2019-12-31,WGDPCHIL Index,278.285,CLP
2,2020-12-31,WGDPCHIL Index,254.042,CLP
3,2021-12-31,WGDPCHIL Index,315.515,CLP
4,2022-12-31,WGDPCHIL Index,302.117,CLP
5,2023-12-31,WGDPCHIL Index,335.533,CLP


In [229]:
GDP_data = {}

for security in securities:
    try:
        df = bquery.bdh([security], fields, start_date=start_date, end_date=end_date_str, options=options)
        if not df.empty:
            currency = currencies[securities.index(security)]
            df.rename(columns={'PX_LAST': 'GDP'}, inplace=True)

            for index, row in df.iterrows():
                
                year = row['date'].year  # Get the year (which is now the index)
                print(year)
                date_str = str(year) + '-12-31'
  # Create date string YYYY1231 for end of year

                if date_str not in GDP_data:
                    GDP_data[date_str] = pd.DataFrame(columns=['Currency', 'GDP'])

                GDP_data[date_str] = pd.concat([GDP_data[date_str], pd.DataFrame({'Currency': [currency], 'GDP': [row['GDP']]})], ignore_index=True)
                

        else:
            print(f"No data retrieved for {security}")

    except Exception as e:
        print(f"Error retrieving data for {security}: {e}")


for date_str in GDP_data:
    GDP_data[date_str] = GDP_data[date_str].set_index('Currency')
    GDP_data[date_str] = GDP_data[date_str].reset_index()

2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023


C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_36584/1478019926.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  GDP_data[date_str] = pd.concat([GDP_data[date_str], pd.DataFrame({'Currency': [currency], 'GDP': [row['GDP']]})], ignore_index=True)
C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_36584/1478019926.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  GDP_data[date_str] = pd.concat([GDP_data[date_str], pd.DataFrame({'Currency': [currency], 'GDP': [row['GDP']]})], ignore_index=True)


2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023


In [230]:
GDP_data['2018-12-31']

,Currency,GDP
0,BRL,1916.934
1,CLP,295.858
2,CNY,13894.907
3,COP,334.198
4,CZK,249.001
5,HUF,160.566
6,IDR,1042.272
7,MYR,358.789
8,MXN,1256.300
9,PEN,222.597


In [372]:
bquery.stop()


In [20]:
from blp import blp 
bquery = blp.BlpQuery().start()
strt_date = '20230101'
end_date = pd.Timestamp('today').strftime('%Y%m%d')
###########################################################################################################################################################
###########################################################################################################################################################
bquery = blp.BlpQuery().start()
trade_balance_tickers = [
    "ECOYBBRN Index", "ECOYBCLN Index", "ECOYBCNN Index", "ECOYBCON Index",
    "ECOYBCZN Index", "ECOYBHUN Index", "ECOYBIDN Index", "ECOYBMYN Index",
    "ECOYBMXN Index", "PRTRBAL Index", "ECOYBPHN Index", "ECOYBPLN Index",
    "ECOYBRON Index", "ECOYBRUN Index", "ECOYBZAN Index", "ECOYBTHN Index",
    "ECOYBTRN Index", "ECOYBINN Index", "ECOYBSGN Index", "ECOYBTWN Index",
    "ECOYBKRN Index", "ECOYBEGN Index","ECOYBEGN Index", "ECOYBEAS Index", "ECOYBJPN Index" 
]

currencies = [
    "BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN",
    "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD",
    "KRW", "UAH", "EGP", "EUR", "JPY"
]

TB_query = bquery.bdh(trade_balance_tickers, ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})
TB_query.rename(columns={'PX_LAST':'TB'}, inplace=True)

# Create conditions dynamically
conditions = [(TB_query['security'] == ticker) for ticker in trade_balance_tickers]

TB_query['Currency'] = np.select(conditions, currencies, default=None) 

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)


In [27]:
TB_query[-3000:-2900]

,date,security,TB,Currency
9500,2024-06-17,ECOYBEGN Index,-34.90,UAH
9501,2024-06-17,ECOYBEAS Index,168.90,EUR
9502,2024-06-17,PRTRBAL Index,1899.32,PEN
9503,2024-06-17,ECOYBEGN Index,-34.90,UAH
9504,2024-06-17,ECOYBTRN Index,-86.90,TRY
...,...,...,...,...
9595,2024-06-20,ECOYBMXN Index,-3.40,MXN
9596,2024-06-20,ECOYBJPN Index,-39.50,JPY
9597,2024-06-20,ECOYBCZN Index,8.10,CZK
9598,2024-06-20,ECOYBTHN Index,-4.80,THB


In [28]:

def create_tb_dict(tb_df):
    """Creates a dictionary of DataFrames with Trade Balance data organized by date.

    Args:
        tb_df: The input DataFrame with 'date', 'security', 'TB', and 'Currency' columns.

    Returns:
        A dictionary where keys are dates (YYYYMMDD strings) and values are DataFrames
        with 'Currency' as index and 'TB' as a column.
    """

    tb_dict = {}
    for date, date_df in tb_df.groupby('date'): #Group by date
        date_str = date.strftime('%Y%m%d')

        # Create a dictionary entry for the date if it doens't exist yet
        if date_str not in tb_dict:
            tb_dict[date_str] = pd.DataFrame()



        # Efficiently creates the DataFrame with "Currency" as index and "TB" as column:
        data = {currency: tb for currency, tb in zip(date_df['Currency'], date_df['TB'])} #uses zip to iterate through lists in parallel
        temp_df = pd.DataFrame.from_dict(data, orient='index', columns=['TB']) #orient = index to have currency as index
        temp_df = temp_df.reindex(currencies)
        temp_df = temp_df.reset_index().rename(columns={'index':'Currency'}) 

        tb_dict[date_str] = temp_df #add or update date in dictionary




    return tb_dict

In [29]:
tb_dict = create_tb_dict(TB_query)


In [30]:
tb_dict.keys()

dict_keys(['20230102', '20230103', '20230104', '20230105', '20230106', '20230109', '20230110', '20230111', '20230112', '20230113', '20230116', '20230117', '20230118', '20230119', '20230120', '20230123', '20230124', '20230125', '20230126', '20230127', '20230130', '20230131', '20230201', '20230202', '20230203', '20230206', '20230207', '20230208', '20230209', '20230210', '20230213', '20230214', '20230215', '20230216', '20230217', '20230220', '20230221', '20230222', '20230223', '20230224', '20230227', '20230228', '20230301', '20230302', '20230303', '20230306', '20230307', '20230308', '20230309', '20230310', '20230313', '20230314', '20230315', '20230316', '20230317', '20230320', '20230321', '20230322', '20230323', '20230324', '20230327', '20230328', '20230329', '20230330', '20230331', '20230403', '20230404', '20230405', '20230406', '20230407', '20230410', '20230411', '20230412', '20230413', '20230414', '20230417', '20230418', '20230419', '20230420', '20230421', '20230424', '20230425', '2023

In [31]:
def calculate_3m_tb_change(tb_dict):
    """Calculates the 3-month Trade Balance change for each DataFrame in the dictionary.

    Args:
        tb_dict: A dictionary where keys are dates ('YYYYMMDD') and values are DataFrames 
                 with 'Currency' as index and 'TB' as a column.

    Returns:
        A dictionary with the same structure as tb_dict, but with an additional 'TB_3m_Chg' column.
    """

    tb_dict_with_change = {}

    for date_str, df in tb_dict.items():  # Iterate through the dictionary
        date = pd.to_datetime(date_str, format='%Y%m%d')  # Convert to datetime object

        # Calculate the date three months prior (using the last day of the month for alignment):
        three_months_ago = (date - pd.DateOffset(months=3)).strftime('%Y%m%d')


        if three_months_ago in tb_dict:  # Only calculate if data for three months ago exists


            df['TB_3m_Chg'] = df['TB'] - tb_dict[three_months_ago]['TB'] # Correct subtraction with aligned indices

        # Handle missing data if 3 months ago date isn't present (fill, 0, etc.)
        else:
            df['TB_3m_Chg'] = np.nan # Or your preferred handling


        tb_dict_with_change[date_str] = df  #Store modified DF in the dict


    return tb_dict_with_change

In [32]:
tb_dict_with_3m_change = calculate_3m_tb_change(tb_dict) #added this line


In [38]:
date_str

NameError: name 'date_str' is not defined

In [349]:


def get_historical_peru_yoy_at_date(security="PRTRBAL Index", date_str=''):
    """Retrieves and calculates the year-over-year trade balance change for Peru at a specific date.

    Args:
        security: The Bloomberg ticker (default: "PRTRBAL Index").
        date_str: The date ('YYYYMMDD').

    Returns:
        The YoY change, or np.nan if error/missing data.
    """
    try:

        date = pd.to_datetime(date_str, format='%Y%m%d')

        # Calculate end-of-month dates relative to the input date
        #end_date_3m = (date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        start_date_3m = date - pd.DateOffset(months=3)


        start_date_1 = start_date_3m.strftime('%Y-%m%d') # YYYY-MM-DD format for BQL
        end_date_1 = end_date_3m.strftime('%Y%m%d')   # YYYY-MM-DD format for BQL
        start_date_2 = (start_date_3m - pd.DateOffset(months=12)).strftime('%Y%m%d')
        end_date_2 = (end_date_3m - pd.DateOffset(months=12)).strftime('%Y%m%d')

        # Construct BQL queries (use lists for securities)
        query_1 = f"get(SUM(px_last(Dates=range({start_date_1}, {end_date_1})))) for({[security]})"
        query_2 = f"get(SUM(px_last(Dates=range({start_date_2}, {end_date_2})))) for({[security]})"

        bql_query_1 = bquery.bql(expression=query_1)
        bql_query_2 = bquery.bql(expression=query_2)


        value_1 = bql_query_1.loc[1, 'value'] # Assumes the result is in the second row
        value_2 = bql_query_2.loc[1, 'value'] # Assumes the result is in the second row


        yoy_change = value_1 - value_2

        return yoy_change

    except Exception as e:
        print(f"Error calculating YoY change for {security} on {date_str}: {e}")  # Include date in error message
        return np.nan



In [51]:
def get_historical_peru_yoy_at_date(tb_dict, date_str): # tb_dict is now an argument
    """Calculates the YoY trade balance change for Peru using data from tb_dict.

    Args:
        tb_dict: Dictionary of Trade Balance DataFrames.
        date_str: The date ('YYYYMMDD').

    Returns:
        The YoY change, or np.nan if error/missing data.
    """
    try:
        date = pd.to_datetime(date_str, format='%Y%m%d')
        #end_date_3m = (date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        start_date_3m = date - pd.DateOffset(months=3)


        # No need for end_date_1 since it is now equal to start_date_1:
        start_date_1 = start_date_3m.strftime('%Y%m%d') # consistent date format
        start_date_2 = (start_date_3m - pd.DateOffset(months=12)).strftime('%Y%m%d')
        print(start_date_1, start_date_2)



        # Get values directly from tb_dict:
        try: #Handle potential KeyError if date is missing

            value_1 = tb_dict[start_date_1].loc['PEN', 'TB'] #access tb_dict directly

        except KeyError:
            print(f"No trade balance data available for {start_date_1}")
            value_1 = np.nan

        try: #Handle potential KeyError if date is missing

            value_2 = tb_dict[start_date_2].loc['PEN', 'TB'] #access tb_dict directly

        except KeyError:

            print(f"No trade balance data available for {start_date_2}")
            value_2 = np.nan



        if pd.isna(value_1) or pd.isna(value_2):  # Handle missing values
            yoy_change = np.nan
        else:

            yoy_change = value_1 - value_2




        return yoy_change

    except Exception as e: #Handle other potential errors.
        print(f"Error calculating YoY change for Peru on {date_str}: {e}")
        return np.nan


# Example usage (updated):

def calculate_historical_peru_yoy(tb_dict):  # Now takes tb_dict as input


    tb_dict_with_yoy = {}

    for date_str in tb_dict: #iterate over existing dates only
        tb_dict_with_yoy[date_str] = tb_dict[date_str].copy()
        try:
            yoy_change = get_historical_peru_yoy_at_date(tb_dict, date_str)  # Pass tb_dict

            tb_dict_with_yoy[date_str].loc['PEN', 'TB_YoY_Chg'] = yoy_change

        except Exception as e:
            print(f"Error in main loop: {e}")


    return tb_dict_with_yoy


#In your script:
tb_dict_with_yoy = calculate_historical_peru_yoy(tb_dict)

20221002 20211002
No trade balance data available for 20221002
No trade balance data available for 20211002
20221003 20211003
No trade balance data available for 20221003
No trade balance data available for 20211003
20221004 20211004
No trade balance data available for 20221004
No trade balance data available for 20211004
20221005 20211005
No trade balance data available for 20221005
No trade balance data available for 20211005
20221006 20211006
No trade balance data available for 20221006
No trade balance data available for 20211006
20221009 20211009
No trade balance data available for 20221009
No trade balance data available for 20211009
20221010 20211010
No trade balance data available for 20221010
No trade balance data available for 20211010
20221011 20211011
No trade balance data available for 20221011
No trade balance data available for 20211011
20221012 20211012
No trade balance data available for 20221012
No trade balance data available for 20211012
20221013 20211013
No trade b

In [54]:
tb_dict['20230829']

,Currency,TB,TB_3m_Chg
0,BRL,74.70,10.0
1,CLP,12.90,2.9
2,CNY,855.80,-53.5
3,COP,-13.00,0.6
4,CZK,-0.70,3.8
5,HUF,0.00,5.8
6,IDR,46.60,-7.0
7,MYR,56.00,1.3
8,MXN,-15.00,11.8
9,PEN,1140.38,-630.2


In [45]:
tb_dict['20241126']

,Currency,TB,TB_3m_Chg
0,BRL,81.10,-14.20
1,CLP,21.50,2.60
2,CNY,929.10,70.40
3,COP,-9.80,-0.40
4,CZK,9.90,1.20
5,HUF,12.80,-0.40
6,IDR,30.10,-1.50
7,MYR,27.50,-4.60
8,MXN,-5.80,-1.90
9,PEN,2342.87,-25.33


In [47]:
print(tb_dict_with_yoy['20241025'])

    Currency       TB  TB_3m_Chg  TB_YoY_Chg
0        BRL    86.00       -9.9         NaN
1        CLP    20.70        2.3         NaN
2        CNY   889.20       36.0         NaN
3        COP    -9.80       -0.5         NaN
4        CZK     9.90        1.3         NaN
5        HUF    12.80       -0.9         NaN
6        IDR    31.10       -1.3         NaN
7        MYR    27.40       -7.1         NaN
8        MXN    -6.60       -2.1         NaN
9        PEN  2342.87     -117.0         NaN
10       PHP   -52.60       -2.6         NaN
11       PLN     4.50       -3.7         NaN
12       RON   -34.70       -1.5         NaN
13       RUB      NaN        NaN         NaN
14       ZAR     6.80       -0.2         NaN
15       THB    -7.10       -2.2         NaN
16       TRY   -78.60        8.9         NaN
17       INR  -260.40       -9.2         NaN
18       SGD    50.00        1.6         NaN
19       TWD    86.40       -3.9         NaN
20       KRW    46.00        7.1         NaN
21       U

In [18]:
tb_dict_with_3m_change['20241126']

,Currency,TB,TB_3m_Chg
0,BRL,81.10,-14.20
1,CLP,21.50,2.60
2,CNY,929.10,70.40
3,COP,-9.80,-0.40
4,CZK,9.90,1.20
5,HUF,12.80,-0.40
6,IDR,30.10,-1.50
7,MYR,27.50,-4.60
8,MXN,-5.80,-1.90
9,PEN,2342.87,-25.33


In [37]:
tb_dict_with_3m_change['20240926']

,Currency,TB,TB_3m_Chg
0,BRL,90.10,-9.50
1,CLP,19.70,1.80
2,CNY,882.60,58.50
3,COP,-9.60,-0.10
4,CZK,9.50,1.40
5,HUF,13.00,-1.10
6,IDR,31.30,-2.20
7,MYR,29.70,-7.30
8,MXN,-7.50,-4.10
9,PEN,2005.07,105.75


In [348]:
def get_peru_trade_balance_yoy_change(security="PRTRBAL Index"):
    """Retrieves and calculates the year-over-year trade balance change for Peru.

    Args:
        security: The Bloomberg ticker for Peru Trade Balance (default: "PRTRBAL Index").

    Returns:
        The year-over-year change as a float, or np.nan if there's an error or missing data.
    """
    try:
        last_update = bquery.bdp([security], ["LAST_UPDATE_DT"]).loc[0, 'LAST_UPDATE_DT']
        eom_last_update = (last_update - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        eom_last_update_3m = (last_update - pd.DateOffset(months=3)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)

        # Format dates for BQL (YYYY-MM-DD)
        start_date_1 = eom_last_update_3m.strftime('%Y-%m-%d')
        end_date_1 = eom_last_update.strftime('%Y-%m-%d')
        start_date_2 = (eom_last_update_3m - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
        end_date_2 = (eom_last_update - pd.DateOffset(months=12)).strftime('%Y-%m-%d')

        # Use lists for securities in BQL queries for consistency:
        query_1 = f"get(SUM(px_last(Dates=range({start_date_1}, {end_date_1})))) for({[security]})" #List format
        query_2 = f"get(SUM(px_last(Dates=range({start_date_2}, {end_date_2})))) for({[security]})" #List format


        bql_query_1 = bquery.bql(expression=query_1)
        bql_query_2 = bquery.bql(expression=query_2)


        value_1 = bql_query_1.loc[1, 'value']  # Correct index 
        value_2 = bql_query_2.loc[1, 'value']  # Correct index

        yoy_change = value_1 - value_2
        return yoy_change

    except Exception as e:  # Handle Bloomberg errors or missing data
        print(f"Error calculating YoY change for {security}: {e}")
        return np.nan

In [268]:
TB_data = {}
# ... (your other code)

all_dates = pd.to_datetime(pd.bdate_range(start=strt_date, end = end_date))
for date in all_dates:
    date_str = date.strftime('%Y%m%d')
    TB_data[date_str] = pd.DataFrame(columns=['Currency', 'TB_3m_chg'])

    for ticker, currency in zip(trade_balance_tickers, currencies):
        TB_data[date_str].loc[currency, 'TB_3m_chg'] = get_historical_3m_chg(ticker, date_str)

Error retrieving 3m change for ECOYBBRN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBCLN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBCNN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBCON Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBCZN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBHUN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBIDN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBMYN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for ECOYBMXN Index on 20241001: can only concatenate list (not "str") to list
Error retrieving 3m change for PRTRBAL Index on 2024100

In [80]:
### VERSION WITH CURRENT DATE WITHOUT LAST UPDATE 
def get_3m_chg(row):
    security = row['security']
    date = row['date']  # Takes security ticker as input
    try:
        LAST_UPDATE = bquery.bdp([f"{security}"], ["LAST_UPDATE_DT"]).loc[0, 'LAST_UPDATE_DT']
        EOM_LAST_UPDATE = (date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        EOM_LAST_UPDATE_3M = (date - pd.DateOffset(months=4)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        start_date = EOM_LAST_UPDATE_3M.strftime('%Y-%m-%d')
        end_date = EOM_LAST_UPDATE.strftime('%Y-%m-%d')
        query_text = f"get(NET_CHG(px_last(Dates=range({start_date}, {end_date}, Frq=M), Fill=PREV))) for(['{security}'])"
        BQL_query = bquery.bql(expression=f"{query_text}")
        return BQL_query.loc[0, 'value']  # Corrected index to 0

    except Exception as e:
        print(f"Error for {security}: {e}")
        return None
TB_query['3m_Chg'] = TB_query.apply(get_3m_chg, axis=1)


In [259]:

def get_3m_chg(row):
    security = row['security']
    date = row['date']

    try:
        # 1. Get historical LAST_UPDATE_DT (this is the key change)
        historical_last_update = bquery.bdh(
            [f"{security}"],  # Ticker
            "LAST_UPDATE_DT", # Field
            date,         # Start date (the date from the row)
            date,         # End date (same as start date to get a single value)
            #Other overrides that might be necessary for the query. For example:
            # overrides =  [("Product","CN")]            
        )

        #Check if a result is given for the given ticker and date, otherwise result will be None
        if historical_last_update.empty:
            return None

        #Extract the last update date for the ticker, for the given date. The result is in the second level of the multiIndex columns
        historical_last_update = historical_last_update.columns.get_level_values(1)[0]


        # 2. Calculate dates relative to the *historical* LAST_UPDATE_DT
        eom_last_update = (historical_last_update - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        eom_last_update_3m = (historical_last_update - pd.DateOffset(months=4)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)


        start_date = eom_last_update_3m.strftime('%Y-%m-%d')
        end_date = eom_last_update.strftime('%Y-%m-%d')

        # 3. BQL query (now using dates based on historical LAST_UPDATE_DT)
        query_text = f"get(NET_CHG(px_last(Dates=range({start_date}, {end_date}, Frq=M), Fill=PREV))) for(['{security}'])"
        bql_query = bquery.bql(expression=f"{query_text}")
        
        #Check if the Bloomberg query returns a result
        if (bql_query.empty):
            return None
        else:
            return bql_query.loc[0, 'value']

    except Exception as e:
        print(f"Error for {security} on {date}: {e}")
        return None


# TB dictionnary is ready 

# List of dictionnaries 



In [63]:

def get_peru_3m_chg(date):
    security = "PRTRBAL Index"
    try:
        # Calculate end-of-month dates relative to the input 'date'
        EOM_date = (date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        EOM_date_3m_ago = (date - pd.DateOffset(months=3)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)

        # Calculate start and end dates for current and previous year periods
        start_date_current = EOM_date_3m_ago.strftime('%Y-%m-%d')
        end_date_current = EOM_date.strftime('%Y-%m-%d')
        start_date_previous = (EOM_date_3m_ago - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
        end_date_previous = (EOM_date - pd.DateOffset(months=12)).strftime('%Y-%m-%d')

        # Bloomberg queries
        BQL_query_current = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_current}, {end_date_current})))) for(['{security}'])")
        BQL_query_previous = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_previous}, {end_date_previous})))) for(['{security}'])")


        #Check if both Bloomberg Queries provide a result, otherwise 3m change will be None
        if (BQL_query_current.empty or BQL_query_previous.empty):
             return None
        else:
            three_month_change = (BQL_query_current.loc[0, 'value'] - BQL_query_previous.loc[0, 'value']) / 1000
            return three_month_change

    except Exception as e:
        print(f"Error for {security} on {date}: {e}")
        return None



start_date = pd.to_datetime('2024-01-01') # Replace with your actual start date
end_date = pd.to_datetime('2024-11-12') # Replace with your actual end date

# Generate a date range
date_range = pd.date_range(start=start_date, end=end_date, freq='B') # 'B' for business days


# Create a DataFrame to store results
peru_trade_balance = pd.DataFrame({'date': date_range})

# Calculate the 3-month change for each date
peru_trade_balance['TB_3m_Chg'] = peru_trade_balance['date'].apply(get_peru_3m_chg)

# ... (Now you can merge peru_trade_balance with your main DataFrame if needed) ...
print(peru_trade_balance)

In [66]:

bquery = blp.BlpQuery().start()

def get_3m_chg(df):
    security = df['security']
    LAST_UPDATE = bquery.bdp( [f"{security}"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
    EOM_LAST_UPDATE = (LAST_UPDATE - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
    EOM_LAST_UPDATE_3M = (LAST_UPDATE - pd.DateOffset(months=4)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
    start_date = EOM_LAST_UPDATE_3M.strftime('%Y-%m-%d')
    end_date = EOM_LAST_UPDATE.strftime('%Y-%m-%d')
    query_text = f"get(NET_CHG(px_last(Dates=range({start_date}, {end_date}, Frq=M), Fill=PREV))) for(['{security}'])"
    BQL_query = bquery.bql(expression=f"{query_text}")
    return BQL_query.loc[1,'value']

TB_query['TB_3m_Chg'] = TB_query.apply(get_3m_chg, axis=1)
TB_query.sort_values(by='Currency', ascending=True, inplace=True)
TB_query.reset_index(inplace=True)
TB_query=TB_query[['Currency', 'security', 'TB', 'TB_3m_Chg']]

###SPECIAL CASE FOR PERU
security = "PRTRBAL Index"
LAST_UPDATE = bquery.bdp( [f"{security}"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
EOM_LAST_UPDATE = (LAST_UPDATE - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
EOM_LAST_UPDATE_3M = (LAST_UPDATE - pd.DateOffset(months=3)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
start_date_1 = EOM_LAST_UPDATE_3M.strftime('%Y-%m-%d')
end_date_1 = EOM_LAST_UPDATE.strftime('%Y-%m-%d')
start_date_2 = (EOM_LAST_UPDATE_3M - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
end_date_2 = (EOM_LAST_UPDATE - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
BQL_query_1 = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_1}, {end_date_1})))) for(['{security}'])")
BQL_query_2 = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_2}, {end_date_2})))) for(['{security}'])")

TB_query.loc[TB_query['Currency'] == 'PEN', 'TB_3m_Chg'] = (BQL_query_1.loc[1,'value']-BQL_query_2.loc[1,'value'])/1000


TBoverGDP = pd.merge(TB_query, GDP_query, on='Currency')
# Find the index of the row where Currency is 'BRL'
brl_index = TBoverGDP[TBoverGDP['Currency'] == 'BRL'].index[0]
################# CUSTOM FOR BRL AND TWD SEE LATER ###########################################################################
###########################################################################

# Divide the TB_3m_Chg_GDP value for 'BRL' by 5.5
TBoverGDP.loc[brl_index, 'TB_3m_Chg'] = -12
# converting to usd for TWD 
twd_index = TBoverGDP[TBoverGDP['Currency'] == 'TWD'].index[0]
TBoverGDP.loc[twd_index, 'GDP'] /= 30.817 # CONVERT TO DOLLARS !! 


TBoverGDP["TB_3m_Chg_GDP"] = TBoverGDP["TB_3m_Chg"]/TBoverGDP["GDP"]
print(TBoverGDP[["GDP",'Currency','TB_3m_Chg']])
TBoverGDP=TBoverGDP[['Currency', 'TB_3m_Chg_GDP']]

# we convert to dollars for BRL 


TBoverGDP

            GDP Currency  TB_3m_Chg
0           NaN      BRL      -12.0
1      2173.666      BRL      -14.1
2      2173.666      BRL      -14.1
3      2173.666      BRL      -14.1
4      2173.666      BRL      -14.1
...         ...      ...        ...
47875   377.782      ZAR       -0.2
47876   377.782      ZAR       -0.2
47877   377.782      ZAR       -0.2
47878   377.782      ZAR       -0.2
47879   377.782      ZAR       -0.2

[47880 rows x 3 columns]


,Currency,TB_3m_Chg_GDP
0,BRL,NaN
1,BRL,-0.006487
2,BRL,-0.006487
3,BRL,-0.006487
4,BRL,-0.006487
...,...,...
47875,ZAR,-0.000529
47876,ZAR,-0.000529
47877,ZAR,-0.000529
47878,ZAR,-0.000529


In [86]:
data_today = pd.ExcelFile('C:/Users/UT3P5T/Desktop/dataframe daily/data_daily2024_11_19.xlsx')
data_today = pd.read_excel(data_today, 'Sheet1',header=0)

In [87]:
data_today

,Currency,MOV_AVG_5D,MOV_AVG_20D,Momentum_x,3M Implied Vol,CPI,REER_x,REER_5Y_avg_x,REER_y,REER_5Y_avg_y,...,size_CN-Agn,size_CN-Awa,size_SN-Agn,size_SN-Awa,size_RR-Agn,size_RR-Awa,size_CR-Agn,size_CR-Awa,size_SR-Agn,size_SR-Awa
0,BRL,5.7756,5.7456,-1,14.1880,0.44000,103.71,107.531905,103.71,107.531905,...,0.000000,0.000000,0.009919,0.000000,0.029757,0.014878,0.019838,0.004959,0.019838,0.004959
1,CLP,976.2200,962.2000,-1,13.4500,0.10000,101.29,102.934762,101.29,102.934762,...,0.020926,0.010463,0.031389,0.010463,0.020926,0.010463,0.000000,0.000000,0.010463,0.005232
2,CNY,7.2325,7.1657,-1,5.9650,0.00000,91.33,98.337302,91.33,98.337302,...,0.047185,0.011796,0.047185,0.011796,0.050000,0.035389,0.047185,0.023592,0.050000,0.023592
3,COP,4438.9200,4381.2500,-1,13.7875,0.24000,105.84,99.467778,105.84,99.467778,...,0.010207,0.005103,0.000000,0.000000,0.000000,0.000000,0.010207,0.005103,0.000000,0.000000
4,CZK,23.9374,23.5715,-1,8.4250,-0.40000,120.15,111.364127,120.15,111.364127,...,0.033407,0.008352,0.000000,0.000000,-0.016704,-0.008352,0.000000,0.000000,-0.016704,-0.016704
5,HUF,385.7000,379.0900,-1,11.4075,-0.10000,108.43,102.842222,108.43,102.842222,...,-0.012336,-0.006168,-0.012336,-0.006168,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,IDR,15833.0000,15740.0000,-1,7.2900,-0.12000,100.19,100.608571,100.19,100.608571,...,0.000000,0.000000,0.019304,0.000000,0.019304,0.019304,0.038609,0.009652,0.038609,0.009652
7,INR,84.3983,84.2170,-1,2.5250,4.52099,99.34,100.289683,99.34,100.289683,...,0.000000,0.000000,-0.050000,-0.050000,-0.050000,-0.050000,-0.050000,-0.027867,-0.050000,-0.050000
8,KRW,1398.8100,1390.0400,-1,9.0150,0.10000,94.78,97.759683,94.78,97.759683,...,0.015610,0.007805,0.031221,0.007805,0.031221,0.015610,0.015610,0.007805,0.015610,0.007805
9,MXN,20.3461,20.1589,-1,14.0550,0.05000,120.51,114.938095,120.51,114.938095,...,0.010013,0.005006,0.000000,0.000000,0.010013,0.005006,0.030038,0.010013,0.010013,0.005006


In [56]:
data_file = pd.ExcelFile('Metrics data.xlsx')
metrics = pd.read_excel(data_file, 'Metrics',header=0)
metrics = metrics.rename(columns={'Unnamed: 0': 'metric'})

In [57]:
metrics

,metric,BRL,CLP,CNY,COP,CZK,HUF,IDR,MYR,MXN,...,TRY,INR,SGD,TWD,KRW,UAH,EGP,EUR,JPY,Avg/Std
0,2021-03-18 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,Momentum,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,NaN
2,REER discount,-0.250451,-0.031295,0.053956,-0.089563,0.064574,-0.036405,-0.018065,-0.021672,0.005603,...,-0.139202,-0.009631,-0.027025,0.031661,-0.015888,0.060914,0.110721,0.014202,-0.038838,NaN
3,Conditional REER,-0.250451,0.000000,0.000000,-0.089563,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.009631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058682
4,3m chg in TB %GDP,0.002699,0.032814,0.012307,-0.001105,0.016712,0.017329,0.013511,0.034717,0.017040,...,-0.007778,0.003195,0.023235,0.000796,0.007143,0.002463,-0.001369,0.002934,0.005457,0.013178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,Conditional REER,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2044,3m chg in TB %GDP,-0.001236,0.007749,0.003956,-0.000275,0.003929,-0.004238,-0.001094,-0.017515,-0.001174,...,0.008123,-0.000845,0.000798,0.001112,0.004087,-0.011021,-0.008840,0.001055,-0.000973,0.006084
2045,Nominal Carry,0.047685,0.003457,-0.009569,0.045088,-0.008943,0.016597,0.016609,-0.015515,0.056870,...,0.371424,0.015297,-0.014292,-0.027122,-0.013682,0.232978,0.180465,-0.015194,-0.043260,0.082164
2046,Real Carry,0.026573,-0.016982,0.013389,0.017551,-0.011075,0.010887,0.025983,-0.007925,0.035737,...,-0.053974,-0.019573,-0.008653,-0.018762,-0.001043,0.156050,-0.043361,-0.009578,-0.043120,0.020172


In [58]:
model_list = pd.read_excel(data_file, 'Model list',header=0)

In [59]:
model_list

,model,Date
0,SR-Awa,2024-11-13
1,SN-Agn,2024-11-12
2,RN-Awa,2024-11-11
3,RN-Awa,2024-11-08
4,RN-Awa,2024-11-07
...,...,...
223,RN-Agn,2024-01-05
224,SN-Awa,2024-01-04
225,CR-Agn,2024-01-03
226,RR-Awa,2024-01-02


In [60]:
def create_date_metric_dict(metrics_df):
    """Creates a dictionary with dates as keys and metric DataFrames as values."""

    date_metric_dict = {}
    i = 0
    while i < len(metrics_df):
        try:
            current_date = pd.to_datetime(metrics_df.iloc[i, 0]).strftime('%Y-%m-%d') 
            metrics_for_date = metrics_df.iloc[i+1:i+8].copy()  
            metrics_for_date = metrics_for_date.set_index('metric') 
            metrics_for_date = metrics_for_date.drop(columns=[c for c in metrics_for_date.columns if c.startswith("Unnamed")])

            date_metric_dict[current_date] = metrics_for_date.transpose()
            i += 7

        except (ValueError, TypeError, IndexError):
            print(f"Skipping invalid row at index {i}")
            i += 1
    return date_metric_dict

date_metric_dictionary = create_date_metric_dict(metrics)


Skipping invalid row at index 7
Skipping invalid row at index 15
Skipping invalid row at index 23
Skipping invalid row at index 31
Skipping invalid row at index 39
Skipping invalid row at index 47
Skipping invalid row at index 55
Skipping invalid row at index 63
Skipping invalid row at index 71
Skipping invalid row at index 79
Skipping invalid row at index 87
Skipping invalid row at index 95
Skipping invalid row at index 103
Skipping invalid row at index 111
Skipping invalid row at index 119
Skipping invalid row at index 127
Skipping invalid row at index 135
Skipping invalid row at index 143
Skipping invalid row at index 151
Skipping invalid row at index 159
Skipping invalid row at index 167
Skipping invalid row at index 175
Skipping invalid row at index 183
Skipping invalid row at index 191
Skipping invalid row at index 199
Skipping invalid row at index 207
Skipping invalid row at index 215
Skipping invalid row at index 223
Skipping invalid row at index 231
Skipping invalid row at ind

In [61]:

def process_metric_dataframes(date_metric_dict):
    """Renames columns and adjusts '3M Implied Vol' within each DataFrame."""
    rows_to_drop = ['UAH', 'EGP', 'EUR', 'JPY', 'RUB']
    columns_to_drop_if_present = ['Conditional_REER_Discount', 'Symmetrical_REER_Discount']  # Columns to drop conditionally

    for date, df in date_metric_dict.items():
        column_mapping = {  # Mapping for column renaming
            'metric' : 'Currency',
            'REER discount': 'REER_Discount_x',
            'Symmetrical REER': 'Symmetrical_REER_Discount',
            '3m chg in TB %GDP': 'TB_3m_Chg_GDP',
            'Nominal Carry': 'Nominal_carry',
            'Real Carry': 'Real_Carry',
            'Implied Volatility (%)': '3M Implied Vol',
            'Conditional REER': 'Conditional_REER_Discount'
        }
        
        df = df.rename(columns=column_mapping, errors='ignore')
        for row_label in rows_to_drop:
            try:  # Handle potential KeyError if label doesn't exist
                df.drop(row_label, axis=0, inplace=True)
            except KeyError:
                print(f"Row '{row_label}' not found in DataFrame for date {date}. Skipping.")  # Informative message

        for col in columns_to_drop_if_present:
            if col in df.columns:
                df.drop(col, axis=1, inplace=True)
                



        if '3M Implied Vol' in df.columns:
            df['3M Implied Vol'] = df['3M Implied Vol'] / 100  



        date_metric_dict[date] = df  


    return date_metric_dict  



def process_metric_dataframes2(date_metric_dict):
    """Renames columns and adjusts '3M Implied Vol' within each DataFrame."""



    for date, df in date_metric_dict.items():


        df_new = df.copy() # Create a copy to avoid modifying the original DataFrame

        df_new = df_new.reset_index(drop=False)  # Reset index
        df_new.columns = ['Currency'] + list(df_new.columns[1:]) #Rename the first column

        date_metric_dict[date] = df_new 
    return date_metric_dict  

In [62]:
date_metric_dictionary = create_date_metric_dict(metrics)
date_metric_dictionary = process_metric_dataframes(date_metric_dictionary) 
date_metric_dictionary = process_metric_dataframes2(date_metric_dictionary) 

Skipping invalid row at index 7
Skipping invalid row at index 15
Skipping invalid row at index 23
Skipping invalid row at index 31
Skipping invalid row at index 39
Skipping invalid row at index 47
Skipping invalid row at index 55
Skipping invalid row at index 63
Skipping invalid row at index 71
Skipping invalid row at index 79
Skipping invalid row at index 87
Skipping invalid row at index 95
Skipping invalid row at index 103
Skipping invalid row at index 111
Skipping invalid row at index 119
Skipping invalid row at index 127
Skipping invalid row at index 135
Skipping invalid row at index 143
Skipping invalid row at index 151
Skipping invalid row at index 159
Skipping invalid row at index 167
Skipping invalid row at index 175
Skipping invalid row at index 183
Skipping invalid row at index 191
Skipping invalid row at index 199
Skipping invalid row at index 207
Skipping invalid row at index 215
Skipping invalid row at index 223
Skipping invalid row at index 231
Skipping invalid row at ind

In [63]:

desired_order = ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'INR', 'KRW', 'MXN', 'MYR', 'PEN', 'PHP', 'PLN', 'RON', 'SGD', 'THB', 'TRY', 'TWD', 'ZAR', 'Avg/Std']


def reorder_currencies(df, desired_order=desired_order):

    # Separate currency rows and 'Avg/Std' row
    currency_rows = df[df['Currency'] != 'Avg/Std']
    avg_std_row = df[df['Currency'] == 'Avg/Std']
    
    # Create a categorical type for the Currency column with the desired order
    currency_rows['Currency'] = pd.Categorical(currency_rows['Currency'], categories=desired_order, ordered=True)
    currency_rows_reordered = currency_rows.sort_values('Currency')
    
    currency_rows_reordered['Currency'] = currency_rows_reordered['Currency'].astype(str) # convert back to string if you need
    
    # Concatenate reordered rows and 'Avg/Std' row
    reordered_df = pd.concat([currency_rows_reordered, avg_std_row], ignore_index=True)
    
    return reordered_df

# Apply the function to each DataFrame
for date, df in date_metric_dictionary.items():
    date_metric_dictionary[date] = reorder_currencies(df, desired_order)


C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_26124/3238837231.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_rows['Currency'] = pd.Categorical(currency_rows['Currency'], categories=desired_order, ordered=True)
C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_26124/3238837231.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_rows['Currency'] = pd.Categorical(currency_rows['Currency'], categories=desired_order, ordered=True)
C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_26124/32388

In [64]:
date_metric_dictionary['2024-11-12']

,Currency,Momentum,REER_Discount_x,TB_3m_Chg_GDP,Nominal_carry,Real_Carry,3M Implied Vol
0,BRL,1.0,-0.059093,-0.000564,0.042643,0.019552,0.152150
1,CLP,0.0,-0.072838,0.007153,0.003765,-0.018665,0.138000
2,CNY,0.0,-0.063990,0.003945,-0.016489,0.004188,0.062325
3,COP,0.0,0.036060,-0.000275,0.043424,0.013911,0.139175
4,CZK,0.0,0.063654,0.003929,-0.009070,-0.013194,0.086650
5,HUF,0.0,0.015728,-0.004238,0.017046,0.009352,0.117600
6,IDR,0.0,-0.007079,-0.000875,0.016209,0.023582,0.073500
7,INR,0.0,0.025557,-0.002563,0.014543,-0.022304,0.026810
8,KRW,0.0,-0.063878,0.004087,-0.014507,-0.003885,0.091500
9,MXN,0.0,0.016279,-0.001174,0.056218,0.033097,0.153250


In [65]:
for date, df in date_metric_dictionary.items():
    df['Momentum'] = df['Momentum'].replace(0, -1)  # Replace 0 with -1 in 'Momentum'
    date_metric_dictionary[date] = df  # Update the dictionary

In [ ]:
#COND and SYM REER DISCOUNT 
REER_discount = pd.merge(REER, Momentum, on='Currency')
REER_discount = REER_discount[['Currency', 'REER_Discount', 'Momentum']]
REER_discount['Conditional_REER_Discount'] = REER_discount.apply(lambda row: row['Momentum'] * row['REER_Discount'] * (row['REER_Discount'] < 0) if row['Momentum'] == 1 else 0, axis=1)
REER_discount['Symmetrical_REER_Discount'] = REER_discount.apply(
    lambda row: row['REER_Discount'] * (np.sign(row['Momentum']) * np.sign(row['REER_Discount']) < 0  ), 
    axis=1
)
REER_discount 

In [66]:
def calculate_symmetrical_reer(df):
    """Calculates Symmetrical REER Discount based on the Excel logic."""

    df['Symmetrical_REER_Discount'] = df.apply(
        lambda row: row['REER_Discount_x'] * (
            1 if (row['Momentum'] == 1 and row['REER_Discount_x'] < 0) or (row['Momentum'] == -1 and row['REER_Discount_x'] > 0)
            else 0
        ),
        axis=1
    )

    
    # Calculate average standard deviation for currencies (excluding 'Avg/Std' row)
    currencies_df = df[df['Currency'] != 'Avg/Std']
    avg_std = currencies_df['Symmetrical_REER_Discount'].std()

    # Update 'Avg/Std' row
    df.loc[df['Currency'] == 'Avg/Std', 'Symmetrical_REER_Discount'] = avg_std

    return df

for date, df in date_metric_dictionary.items():
    date_metric_dictionary[date] = calculate_symmetrical_reer(df)

In [67]:
def add_conditional_reer(df):
    """
    Adds 'Conditional_REER_Discount' column if it doesn't exist.
    """
    if 'Conditional_REER_Discount' not in df.columns:
        df['Conditional_REER_Discount'] = df.apply(
            lambda row: row['Momentum'] * row['REER_Discount_x'] * (row['REER_Discount_x'] < 0) if row['Momentum'] == 1 else 0, 
            axis=1
        )
    return df


# Apply the function to each DataFrame in the dictionary
for date, df in date_metric_dictionary.items():
    date_metric_dictionary[date] = add_conditional_reer(df)

In [68]:
### ADAPTED Z SCORE FUNCTION #######
models = {  # Dictionary mapping short model names to criteria lists
    "RN": ["REER_Discount_x", "TB_3m_Chg_GDP", "Nominal_carry"],
    "CN": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "SN": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "RR": ["REER_Discount_x", "TB_3m_Chg_GDP", "Real_Carry"],
    "CR": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
    "SR": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
}
def calculate_zscore(df, criteria):
    """Calculates the z-score based on the provided criteria."""
    zscore_parts = []

    for i, criterion in enumerate(criteria):
        if df[criterion].std() == 0:  # Handle zero standard deviation
            zscore_part = 0
        elif criterion in ["Symmetrical_REER_Discount", "Conditional_REER_Discount"]:
            zscore_part = -df[criterion] / df[criterion].std()
        else:
            zscore_part = df[criterion] / df[criterion].std()

        zscore_parts.append(zscore_part)

    # Return NA for the 'Avg/Std' row
    return np.where(df['Currency'] == 'Avg/Std', np.nan, sum(zscore_parts)/3)


# Loop through each date in the dictionary
for date, df in date_metric_dictionary.items():
    print(date)
    # Find the recommended model for the current date
    model_row = model_list[model_list['Date'] == date]
    
    if not model_row.empty:
        model_name = model_row['model'].iloc[0][:2]  # Extract first two characters (e.g., "SN", "RN")
        criteria = models[model_name]  # Get criteria for the model


        df['z_score'] = calculate_zscore(df, criteria)



2021-03-18
2021-04-01
2021-05-03
2021-06-01
2021-07-01
2021-08-02
2021-09-01
2021-10-01
2021-11-01
2021-12-01
2022-01-03
2022-02-01
2022-03-01
2022-04-01
2022-05-02
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-03
2022-11-01
2022-12-01
2023-01-03
2023-02-01
2023-03-01
2023-04-03
2023-05-01
2023-06-01
2023-07-03
2023-08-01
2023-09-01
2023-10-02
2023-11-01
2023-12-01
2024-01-02
2024-01-03
2024-01-04
2024-01-05
2024-01-08
2024-01-09
2024-01-10
2024-01-11
2024-01-12
2024-01-15
2024-01-16
2024-01-17
2024-01-18
2024-01-19
2024-01-22
2024-01-23
2024-01-24
2024-01-25
2024-01-26
2024-01-29
2024-01-30
2024-01-31
2024-02-01
2024-02-02
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-19
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-26
2024-02-27
2024-02-29
2024-03-01
2024-03-04
2024-03-05
2024-03-06
2024-03-07
2024-03-08
2024-03-11
2024-03-12
2024-03-13
2024-03-14
2024-03-15
2024-03-18
2024-03-19
2024-03-20
2024-03-21

In [66]:
model_list['Date'].values

array(['2024-11-13T00:00:00.000000000', '2024-11-12T00:00:00.000000000',
       '2024-11-11T00:00:00.000000000', '2024-11-08T00:00:00.000000000',
       '2024-11-07T00:00:00.000000000', '2024-11-06T00:00:00.000000000',
       '2024-11-05T00:00:00.000000000', '2024-11-04T00:00:00.000000000',
       '2024-11-01T00:00:00.000000000', '2024-10-31T00:00:00.000000000',
       '2024-10-30T00:00:00.000000000', '2024-10-29T00:00:00.000000000',
       '2024-10-28T00:00:00.000000000', '2024-10-25T00:00:00.000000000',
       '2024-10-24T00:00:00.000000000', '2024-10-23T00:00:00.000000000',
       '2024-10-22T00:00:00.000000000', '2024-10-21T00:00:00.000000000',
       '2024-10-18T00:00:00.000000000', '2024-10-17T00:00:00.000000000',
       '2024-10-16T00:00:00.000000000', '2024-10-15T00:00:00.000000000',
       '2024-10-14T00:00:00.000000000', '2024-10-11T00:00:00.000000000',
       '2024-10-10T00:00:00.000000000', '2024-10-09T00:00:00.000000000',
       '2024-10-08T00:00:00.000000000', '2024-10-07

In [69]:
import pandas as pd
import numpy as np
models = {
    "RN": ["REER_Discount_x", "TB_3m_Chg_GDP", "Nominal_carry"],
    "CN": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "SN": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "RR": ["REER_Discount_x", "TB_3m_Chg_GDP", "Real_Carry"],
    "CR": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
    "SR": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
}

def round_2x_to_half(x):
    two_x = 2 * x
    return round(two_x * 2) / 2 if two_x >= 0 else -np.round(-two_x * 2) / 2

# Conviction mapping dictionaries (define outside the function)
conviction_mapping_agnostic = {  # No change here
    0: 3, 1: 3, 2: 2, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1,
    8: 0, 9: 0, 10: 0, 11: 0, 12: -1, 13: -1, 14: -1,
    15: -2, 16: -2, 17: -2, 18: -3, 19: -3
}


def calculate_conviction_and_zscore(df, model_name):
    """Calculates both conviction and z-score, handling 'Avg/Std' row and NaNs."""

    model_type = model_name.split("-")[1]
    short_model_name = model_name[:2]
    criteria = models.get(short_model_name)

    if criteria:
        # Filter out "Avg/Std" *before* calculations:
        df_currencies = df[df['Currency'] != 'Avg/Std'].copy()  #Important: create a copy to avoid SettingWithCopyWarning



        # 1. Calculate Z-score:
        df_currencies[f'zscore_{short_model_name}'] = calculate_zscore(df_currencies, criteria)

        # 2. Calculate Rank (and fill NaNs if any):
        df_currencies[f'rank_{short_model_name}'] = df_currencies[f'zscore_{short_model_name}'].rank(ascending=False, method='dense').fillna(-1).astype(int)


        # 3. Calculate Conviction:
        if model_type == 'Agn':
            df_sorted = df_currencies.sort_values(by=f'rank_{short_model_name}')

            df_sorted[f'conviction_{model_name}'] = df_sorted[f'rank_{short_model_name}'].apply(
                 lambda rank: conviction_mapping_agnostic.get(int(rank) - 1, np.nan) if not pd.isna(rank) else None)

            df = df.merge(df_sorted[[f'conviction_{model_name}', 'Currency']], on='Currency', how='left') #Important: add currency to merge on

        elif model_type == 'Awa':

            df_currencies[f'conviction_{model_name}'] = df_currencies.apply(
                lambda row: _calculate_awa_conviction(row, short_model_name), axis=1
             )
            df = df.merge(df_currencies[[f'conviction_{model_name}', 'Currency']], on='Currency', how='left')#Merge the result back


    return df

def _calculate_awa_conviction(row, model_name): #Correct the ranges 
    try:
        rank = int(row[f'rank_{model_name}'])
    except (ValueError, TypeError):  #Catching more types of errors
        return np.nan  # Or another default value as needed


    zscore = row[f'zscore_{model_name}']
    if rank <= 8:
        conviction = max(-3, min(3, np.round(zscore * 4)/2 ))
    elif rank <=12: #Between 9 and 12
        conviction=0
    elif rank >= 13 :# >= 13

        conviction = max(-3, min(3, round_2x_to_half(zscore)))
    else:
        return np.nan
    return conviction 




# Main loop:
# Ensure 'Date' in model_list is datetime64[ns] type:
model_list['Date'] = model_list['Date'].dt.strftime('%Y-%m-%d')

for date_str, df in date_metric_dictionary.items():
    #Ensure also that your dictionnary keys are in datetime format, without time 
    date = pd.to_datetime(date_str).strftime('%Y-%m-%d')#Remove time from timestamp
    
    model_row = model_list[model_list['Date'] == date]

    if not model_row.empty:

        model_name = model_row['model'].iloc[0]
        date_metric_dictionary[date_str] = calculate_conviction_and_zscore(df.copy(), model_name) 

    else:
        print(f"No model found for {date}.")


No model found for 2021-03-18.
No model found for 2021-04-01.
No model found for 2021-05-03.
No model found for 2021-06-01.
No model found for 2021-07-01.
No model found for 2021-08-02.
No model found for 2021-09-01.
No model found for 2021-10-01.
No model found for 2021-11-01.
No model found for 2021-12-01.
No model found for 2022-01-03.
No model found for 2022-02-01.
No model found for 2022-03-01.
No model found for 2022-04-01.
No model found for 2022-05-02.
No model found for 2022-06-01.
No model found for 2022-07-01.
No model found for 2022-08-01.
No model found for 2022-09-01.
No model found for 2022-10-03.
No model found for 2022-11-01.
No model found for 2022-12-01.
No model found for 2023-01-03.
No model found for 2023-02-01.
No model found for 2023-03-01.
No model found for 2023-04-03.
No model found for 2023-05-01.
No model found for 2023-06-01.
No model found for 2023-07-03.
No model found for 2023-08-01.
No model found for 2023-09-01.
No model found for 2023-10-02.
No model

In [79]:
date_metric_dictionary['2024-11-05']

,Currency,Momentum,REER_Discount_x,TB_3m_Chg_GDP,Nominal_carry,Real_Carry,3M Implied Vol,Symmetrical_REER_Discount,Conditional_REER_Discount,z_score,conviction_RN-Awa,size_0_RN-Awa,size_1_RN-Awa
0,BRL,-1.0,-0.076716,-0.001597,0.043335,0.023636,0.163550,-0.000000,0.000000,-0.346289,-0.5,-0.004721,-0.004721
1,CLP,-1.0,-0.058377,0.006855,0.001938,-0.014725,0.139900,-0.000000,0.000000,0.047266,0.0,0.000000,0.000000
2,CNY,-1.0,-0.061588,0.002012,-0.022211,-0.002790,0.066025,-0.000000,0.000000,-0.327895,-0.5,-0.011694,-0.011694
3,COP,-1.0,0.028354,-0.000275,0.044379,0.010946,0.143825,0.028354,0.000000,0.322186,0.5,0.005369,0.005369
4,CZK,-1.0,0.069783,0.004534,-0.009126,-0.011289,0.084650,0.069783,0.000000,0.601642,1.0,0.018243,0.018243
5,HUF,-1.0,0.022056,-0.004238,0.018304,0.012631,0.113925,0.022056,0.000000,-0.030675,0.0,0.000000,0.000000
6,IDR,-1.0,-0.021303,-0.000875,0.016245,0.023623,0.083000,-0.000000,0.000000,-0.102609,-0.0,-0.000000,-0.000000
7,INR,-1.0,0.010670,-0.002563,0.016324,-0.013727,0.025750,0.010670,0.000000,-0.012704,0.0,0.000000,0.000000
8,KRW,-1.0,-0.060679,0.004204,-0.015596,-0.005009,0.106000,-0.000000,0.000000,-0.178375,-0.5,-0.007284,-0.007284
9,MXN,-1.0,0.025641,-0.001174,0.057344,0.036042,0.189950,0.025641,0.000000,0.310170,0.5,0.004065,0.004065


In [71]:
########## DATA PRICES 
from xbbg import blp 
import pandas as pd
from dateutil.relativedelta import relativedelta

# Define the list of Bloomberg tickers
tickers = ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy",
           "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy",
           "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy",
           "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy",
           "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy",
           "KRW CMPN Curncy"]

# Extract currency codes from tickers
currencies = [ticker[:3] for ticker in tickers]

# Get historical data for the past 5 years
end_date = pd.Timestamp.today()
start_date = end_date - relativedelta(years=5)

data_prices = blp.bdh(tickers, 'PX_LAST', start_date, end_date)
data_prices.columns = currencies
data_prices

returns = data_prices.shift()/data_prices -1 
returns = returns.dropna()
returns = returns.drop('RUB', axis=1).copy()

In [72]:
# Define models and constants
models = ["RN", "CN", "SN", "RR", "CR", "SR"]
concentration_limit = 0.05
carry_cap = -0.003
lower_te_bound = 0.008
upper_te_bound = 0.012
# These values were missing, I have added them based on the 'correct' code.
i2_value = -0.003
k2_value = 0.005
ab38_value = 6
default = 0.03
model_list['Date'] = pd.to_datetime(model_list['Date']).dt.strftime('%Y-%m-%d')

for date_str, df in date_metric_dictionary.items():
    date = pd.to_datetime(date_str).strftime('%Y-%m-%d')
    model_row = model_list[model_list['Date'] == date]
    if not model_row.empty:
        model_name_method = model_row['model'].iloc[0]  # Get the full "CN-Agn" format
        model_name = model_name_method[:2]
        method = model_name_method[3:]  # Extracts method

        # Calculate average 3M Implied Vol *per DataFrame*
        avg_implied_vol = df['3M Implied Vol'].mean()

        conviction_col = f"conviction_{model_name_method}"  # Use model_name_method!



        standard_size = 0.03 if method == "Agn" else 0.035  # Method correctly extracted

        # Initial size
        df[f'size_0_{model_name_method}'] = df.apply(  # model_name_method here
             lambda row: (
                i2_value / row['Nominal_carry']
                if (
                    max(
                        min(
                            avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * ab38_value * k2_value / 2,
                            concentration_limit
                        ),
                        -concentration_limit
                    ) * row['Nominal_carry'] < i2_value
                )
                else (
                    max(
                        min(
                            avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * ab38_value * k2_value / 2,
                            concentration_limit
                        ),
                        -concentration_limit
                    )
                )
            ),
            axis=1
        )

        # Carry Control:
        df[f'size_1_{model_name_method}'] = df.apply(
             lambda row: row[f'size_0_{model_name_method}'] # Corrected
            if row[f'size_0_{model_name_method}'] * row['Nominal_carry'] >= carry_cap
            else carry_cap / row['Nominal_carry'],
            axis=1
        )

        # store DataFrame back into the dictionary if it has been modified
        date_metric_dictionary[date_str] = df 

    else:
        print(f"No model found for {date}.")


No model found for 2021-03-18.
No model found for 2021-04-01.
No model found for 2021-05-03.
No model found for 2021-06-01.
No model found for 2021-07-01.
No model found for 2021-08-02.
No model found for 2021-09-01.
No model found for 2021-10-01.
No model found for 2021-11-01.
No model found for 2021-12-01.
No model found for 2022-01-03.
No model found for 2022-02-01.
No model found for 2022-03-01.
No model found for 2022-04-01.
No model found for 2022-05-02.
No model found for 2022-06-01.
No model found for 2022-07-01.
No model found for 2022-08-01.
No model found for 2022-09-01.
No model found for 2022-10-03.
No model found for 2022-11-01.
No model found for 2022-12-01.
No model found for 2023-01-03.
No model found for 2023-02-01.
No model found for 2023-03-01.
No model found for 2023-04-03.
No model found for 2023-05-01.
No model found for 2023-06-01.
No model found for 2023-07-03.
No model found for 2023-08-01.
No model found for 2023-09-01.
No model found for 2023-10-02.
No model

In [80]:
date_metric_dictionary['2024-11-07']

,Currency,Momentum,REER_Discount_x,TB_3m_Chg_GDP,Nominal_carry,Real_Carry,3M Implied Vol,Symmetrical_REER_Discount,Conditional_REER_Discount,z_score,conviction_RN-Awa,size_0_RN-Awa,size_1_RN-Awa
0,BRL,-1.0,-0.057185,-0.001597,0.041880,0.022176,0.148100,-0.000000,0.000000,-0.224777,-0.5,-0.004695,-0.004695
1,CLP,-1.0,-0.048055,0.007153,0.002083,-0.014578,0.136500,-0.000000,0.000000,0.115217,0.0,0.000000,0.000000
2,CNY,-1.0,-0.061341,0.003945,-0.020459,-0.000966,0.061875,-0.000000,0.000000,-0.225270,-0.5,-0.011237,-0.011237
3,COP,-1.0,0.054942,-0.000275,0.044008,0.010578,0.136475,0.054942,0.000000,0.483821,1.0,0.010189,0.010189
4,CZK,-1.0,0.069955,0.003929,-0.008913,-0.011070,0.078950,0.069955,0.000000,0.561222,1.0,0.017614,0.017614
5,HUF,-1.0,0.031814,-0.004238,0.016973,0.011278,0.107475,0.031814,0.000000,0.028991,0.0,0.000000,0.000000
6,IDR,-1.0,-0.015876,-0.000875,0.016850,0.024242,0.073370,-0.000000,0.000000,-0.062600,-0.0,-0.000000,-0.000000
7,INR,-1.0,0.013202,-0.002563,0.014517,-0.015516,0.025960,0.013202,0.000000,0.001898,0.0,0.000000,0.000000
8,KRW,-1.0,-0.051276,0.004204,-0.014301,-0.003673,0.088380,-0.000000,0.000000,-0.128591,-0.5,-0.007867,-0.007867
9,MXN,-1.0,0.051410,-0.001174,0.054267,0.031151,0.147220,0.051410,0.000000,0.460918,1.0,0.009446,0.009446


In [81]:
### CHECK WHY MULTIPLY BY 100 TO OBTAIN THE RIGHT RESULT 
for model_name in models:
    for method in ["Agn", "Awa"]:
        model = f"{model_name}-{method}"
        conviction_col = f"conviction_{model_name}_{method}"
        standard_size = 0.03 if method == "Agn" else 0.035 
        implied_vol_sum = df['3M Implied Vol'].sum()

        for u in range(2, 7):  # u in [2, 3, 4, 5, 6]  (Steps 2-6)
            standard_size = 0.03 if method == "Agn" else 0.035  
            size_prev_col = f'size_{u-1}_{model}'
            size_curr_col = f'size_{u}_{model}'
            weights = df[size_prev_col]

            portfolio_returns = np.dot(returns.values, weights)
            annualized_std = np.std(portfolio_returns) * np.sqrt(252)
            df[f'std_{model}_step{u - 1}'] = annualized_std

            ex_ante_te = annualized_std

            if ex_ante_te < lower_te_bound and ex_ante_te != 0:
                standard_size *= math.ceil(lower_te_bound / ex_ante_te) #Update standard size iteratively
            elif ex_ante_te > upper_te_bound:
                standard_size *= math.floor(upper_te_bound / ex_ante_te)
            elif ex_ante_te == 0:  # Handle the zero case if needed.
                standard_size *= 1 # No change to standard size in this case


            df[size_curr_col] = 100*df.apply(
                lambda row: (
                    carry_cap / row['Nominal_carry']
                    if (
                        max(
                            min(
                                avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * standard_size * k2_value / 2,
                                concentration_limit
                            ),
                            -concentration_limit
                        ) * row['Nominal_carry'] < carry_cap
                    )
                    else (
                        max(
                            min(
                                avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * standard_size * k2_value / 2,
                                concentration_limit
                            ),
                            -concentration_limit
                        )
                    )
                ),
                axis=1
            )

            df[size_curr_col] = df.apply(
                lambda row: (
                    row[size_curr_col]
                    if row[size_curr_col] * row['Nominal_carry'] >= carry_cap
                    else carry_cap / row['Nominal_carry']
                ),
                axis=1
            )
            df[size_curr_col]  = df[size_curr_col].clip(lower=-concentration_limit, upper=concentration_limit)


    ### TAKE THE RESULT AND BOUND BETWEEN -5% and 5% 



KeyError: 'size_1_RN-Agn'

In [83]:

def calculate_sizes(date_metric_dictionary, model_list, returns): # added returns as argument
    """Calculates position sizes based on various factors.

    Args:
        date_metric_dictionary (dict): Dictionary of DataFrames.
        model_list (pd.DataFrame): DataFrame containing model information.
        returns (pd.DataFrame): DataFrame of returns used for TE calculation.

    Returns:
         dict: Updated date_metric_dictionary.
    """




    for date_str, df in date_metric_dictionary.items():
        date = pd.to_datetime(date_str).strftime('%Y-%m-%d')


        model_row = model_list[model_list['Date'] == date]
        if not model_row.empty:
            model_name_method = model_row['model'].iloc[0]
            model_name = model_name_method[:2]
            method = model_name_method[3:]

            avg_implied_vol = df['3M Implied Vol'].mean()


            for model_name in models:
                for method in ["Agn", "Awa"]:
                    model = f"{model_name}-{method}"
                    conviction_col = f"conviction_{model}"  # Correct column naming



                    standard_size = 0.03 if method == "Agn" else 0.035 #reinitialize standard_size within the loop



                    for u in range(2, 7):
                        size_prev_col = f'size_{u-1}_{model}'
                        size_curr_col = f'size_{u}_{model}'
                        weights = df[size_prev_col]


                        portfolio_returns = np.dot(returns.values, weights) # use returns argument. Make sure it has the correct index alignment with your weights.
                        annualized_std = np.std(portfolio_returns) * np.sqrt(252)
                        df[f'std_{model}_step{u - 1}'] = annualized_std

                        ex_ante_te = annualized_std

                        if ex_ante_te < lower_te_bound and ex_ante_te != 0:
                            standard_size *= math.ceil(lower_te_bound / ex_ante_te)
                        elif ex_ante_te > upper_te_bound:
                            standard_size *= math.floor(upper_te_bound / ex_ante_te)
                        elif ex_ante_te == 0:
                            standard_size *= 1



                        df[size_curr_col] = df.apply( #removed "*100"
                            lambda row: (
                                # ... (rest of your lambda function – no changes needed)
                            ),
                            axis=1
                        )

                        df[size_curr_col] = df.apply( # carry control and clipping - no changes
                            lambda row: (
                              # ...
                            ),
                            axis=1
                        )
                        df[size_curr_col] = df[size_curr_col].clip(lower=-concentration_limit, upper=concentration_limit)

        # Store updated DataFrame back in the dictionary (if there is a model for the date)
        if not model_row.empty: # Only update dict if model exists for date
            date_metric_dictionary[date_str] = df


        else:
            print(f'No model for {date}')

    return date_metric_dictionary




# Example usage
# ... create your date_metric_dictionary, model_list, and returns dataframes
updated_date_metric_dictionary = calculate_sizes(date_metric_dictionary, model_list, returns) # pass returns to the function

No model for 2021-03-18
No model for 2021-04-01
No model for 2021-05-03
No model for 2021-06-01
No model for 2021-07-01
No model for 2021-08-02
No model for 2021-09-01
No model for 2021-10-01
No model for 2021-11-01
No model for 2021-12-01
No model for 2022-01-03
No model for 2022-02-01
No model for 2022-03-01
No model for 2022-04-01
No model for 2022-05-02
No model for 2022-06-01
No model for 2022-07-01
No model for 2022-08-01
No model for 2022-09-01
No model for 2022-10-03
No model for 2022-11-01
No model for 2022-12-01
No model for 2023-01-03
No model for 2023-02-01
No model for 2023-03-01
No model for 2023-04-03
No model for 2023-05-01
No model for 2023-06-01
No model for 2023-07-03
No model for 2023-08-01
No model for 2023-09-01
No model for 2023-10-02
No model for 2023-11-01
No model for 2023-12-01


KeyError: 'size_1_RN-Agn'

In [ ]:
updated_date_metric_dictionary = calculate_sizes(date_metric_dictionary)